![alt text](./Cerny_logo_1.jpg)

# Volume targeted ventilation during neonatal transport

Code supplement to this paper: Lantos, L., Berenyi, A., Morley, C., Somogyvari Zs & Belteki G. **Volume guarantee ventilation in neonates treated with hypothermia for hypoxic-ischemic encephalopathy during interhospital transport.** J Perinatol (2020). https://doi.org/10.1038/s41372-020-00823-8

**Author: Dr Gusztav Belteki**

### Import the required libraries and set options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt

import os
import sys
import re
import pickle
import datetime

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta

%matplotlib inline

matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_colwidth', -1)

pd.options.mode.chained_assignment = None  # default='warn'

# This suppresses the legend warning
import logging
logging.getLogger().setLevel(logging.CRITICAL)

In [ ]:
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))
print("IPython version: {}".format(IPython.__version__))

### List and set the working directory and the directories to export results and graphs

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'fabian'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = '/Users/guszti/ventilation_fabian'

DIR_WRITE = '%s/%s/%s' % (CWD, 'Analyses', 'Analysis_HIE')
if not os.path.isdir(DIR_WRITE):
    os.makedirs(DIR_WRITE)

# Images and raw data will be written on an external hard drive
if not os.path.isdir('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)):
    os.makedirs('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC))
DATA_DUMP = '/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)

### Import clinical data of the recordings with possible HIE diagnosis

As this list was prepared from ICH codes, not all these infants may have had HIE and therapeutic hypothermia

In [ ]:
with open('%s/%s.pickle' % (DATA_DUMP, 'clin_df_HIE'), 'rb') as handle:
    clin_df_HIE = pickle.load(handle)

In [ ]:
len(clin_df_HIE)

In [ ]:
clin_df_HIE.head()

### Identify recordings originating from the same infants

In [ ]:
clin_df_HIE[clin_df_HIE.duplicated(subset = ['Date of Birth', 'Gestational Age (weeks)', 'Birth Weight'], 
                                   keep = False)]

### Import ventilator data from pickle archives

In [ ]:
# Import ventilator parameters, settings and alarms

with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_measurements_ventilated_1_300'), 'rb') as handle:
    data_pars_measurements_ventilated_1_300 = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_settings_ventilated_1_300'), 'rb') as handle:
    data_pars_settings_ventilated_1_300 = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_alarms_ventilated_1_300'), 'rb') as handle:
    data_pars_alarms_ventilated_1_300 = pickle.load(handle)
    

with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_measurements_ventilated_301_600'), 'rb') as handle:
    data_pars_measurements_ventilated_301_600 = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_settings_ventilated_301_600'), 'rb') as handle:
    data_pars_settings_ventilated_301_600 = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_alarms_ventilated_301_600'), 'rb') as handle:
    data_pars_alarms_ventilated_301_600 = pickle.load(handle)
                            

with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_measurements_ventilated_601_665'), 'rb') as handle:
    data_pars_measurements_ventilated_601_900 = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_settings_ventilated_601_665'), 'rb') as handle:
    data_pars_settings_ventilated_601_900 = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_alarms_ventilated_601_665'), 'rb') as handle:
    data_pars_alarms_ventilated_601_900 = pickle.load(handle)
    
    
data_pars_measurements_ventilated = {**data_pars_measurements_ventilated_1_300, 
                                     **data_pars_measurements_ventilated_301_600,
                                     **data_pars_measurements_ventilated_601_900}

data_pars_settings_ventilated = {**data_pars_settings_ventilated_1_300, 
                                 **data_pars_settings_ventilated_301_600,
                                 **data_pars_settings_ventilated_601_900}

data_pars_alarms_ventilated = {**data_pars_alarms_ventilated_1_300, 
                               **data_pars_alarms_ventilated_301_600,
                               **data_pars_alarms_ventilated_601_900}

In [ ]:
# Import DataFrames with ventilation modes

with open('%s/%s.pickle' % (DATA_DUMP, 'vent_modes_ventilated_1_300'), 'rb') as handle:
    vent_modes_ventilated_1_300 = pickle.load(handle)

with open('%s/%s.pickle' % (DATA_DUMP, 'vent_modes_ventilated_301_600'), 'rb') as handle:
    vent_modes_ventilated_301_600 = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'vent_modes_ventilated_601_665'), 'rb') as handle:
    vent_modes_ventilated_601_900 = pickle.load(handle)
    
vent_modes_ventilated = pd.concat([vent_modes_ventilated_1_300, vent_modes_ventilated_301_600,
                                   vent_modes_ventilated_601_900], sort = False)

In [ ]:
len(data_pars_measurements_ventilated), len(vent_modes_ventilated)

### Import blood gases

In [ ]:
with open('%s/%s.pickle' % (DATA_DUMP, 'blood_gases_1_665'), 'rb') as handle:
    blood_gases = pickle.load(handle)

### How many of the selected infants were ventilated for longer than 15 minutes

In [ ]:
# These are the babies who were not ventilated for >15 minutes

print(set(clin_df_HIE.index)  - set(vent_modes_ventilated.index))

In [ ]:
recordings = sorted(clin_df_HIE.index & vent_modes_ventilated.index)
len(recordings)

Select only those recordings which are ventilated for >15 minutes and HIE

In [ ]:
data_pars_measurements_ventilated_HIE = {rec: data_pars_measurements_ventilated[rec] for rec 
                                         in data_pars_measurements_ventilated if rec in recordings}

data_pars_settings_ventilated_HIE = {rec: data_pars_settings_ventilated[rec] for rec 
                                         in data_pars_settings_ventilated if rec in recordings}

data_pars_alarms_ventilated_HIE = {rec : data_pars_alarms_ventilated[rec] for rec 
                                         in data_pars_alarms_ventilated if rec in recordings}

vent_modes_ventilated_HIE = vent_modes_ventilated.reindex(recordings)

In [ ]:
len(data_pars_measurements_ventilated_HIE)

In [ ]:
print(recordings)

Trim the clinical recordings to the selected dataset

In [ ]:
clin_df_HIE = clin_df_HIE.loc[recordings]

In [ ]:
len(clin_df_HIE)

### Only keep relevant cases

Exclude infants born at <36 weeks gestation

In [ ]:
clin_df_HIE = clin_df_HIE[clin_df_HIE['Gestational Age (weeks)'] >= 36]
len(clin_df_HIE)

Exclude infants who were transferred after the first day of life

In [ ]:
clin_df_HIE = clin_df_HIE[clin_df_HIE['Postnatal Age'] < pd.to_timedelta('1D')]
len(clin_df_HIE)

Exclude infants that have been miscoded and they were not HIE or who did not undergo therapeutic hypothermia This is based on manual inspection of case notes (Lajos Lantos) rather than just parsing ICD codes

- AL000056 – Légzészavar. No cooling
- AL000188 – Légzészavar. No cooling
- AL000219 – Intubálási szövődmény, nem volt asphyxia. No cooling.
- AL000277 – No HIE, no criteria met, no cooling
- AL000592 – Légzészavar, no cooling      

In [ ]:
clin_df_HIE = clin_df_HIE.drop(['AL000056', 'AL000188', 'AL000219', 'AL000277', 'AL000592',])
len(clin_df_HIE)

Exclude infants who had major congenital malformations

In [ ]:
clin_df_HIE[['Pathology_English']];

This list has been manually curated after reviewing the case notes directly (Lajos Lantos) rather than just relying on ICD codes.

 - AL000030: Polycystic kidney. EXCLUDE
 - AL000113: anorectal malformation, cleft palate and complex congenital heart disease. EXCLUDE
 - AL000612: Minor anomalies: polydactilia, syndactilia. No major anomaly. CAN BE INCLUDED

In [ ]:
clin_df_HIE = clin_df_HIE.drop(['AL000030',  'AL000113', ])
len(clin_df_HIE)

Exclude an atypical case: "20 órás korban anya mellett leszürkül, cyanotikus, resus needed. Nem a klasszikus HIE. De hűtve volt." 

In [ ]:
clin_df_HIE = clin_df_HIE.drop(['AL000625',])
len(clin_df_HIE)

In [ ]:
recordings = sorted(clin_df_HIE.index)
len(recordings)

### Combine recordings which are from the same infants

`AL000443` and `AL000450` recordings are from the same infant (consecutive recording). Combine them.

In [ ]:
data_pars_alarms_ventilated_HIE['AL000443'] = pd.concat([data_pars_alarms_ventilated_HIE['AL000443'],
                                                        data_pars_alarms_ventilated_HIE['AL000450']], sort=True)
data_pars_measurements_ventilated_HIE['AL000443'] = pd.concat([data_pars_measurements_ventilated_HIE['AL000443'],
                                                        data_pars_measurements_ventilated_HIE['AL000450']], sort=True)

data_pars_settings_ventilated_HIE['AL000443'] = pd.concat([data_pars_settings_ventilated_HIE['AL000443'],
                                                        data_pars_settings_ventilated_HIE['AL000450']], sort=True)

del data_pars_measurements_ventilated_HIE['AL000450']
del data_pars_settings_ventilated_HIE['AL000450']
del data_pars_alarms_ventilated_HIE['AL000450']

In [ ]:
vent_modes_ventilated_HIE.loc[['AL000443', 'AL000450']]

In [ ]:
vent_modes_ventilated_HIE.loc['AL000443', 'SIPPV'] = vent_modes_ventilated_HIE.loc['AL000450']['SIPPV']
vent_modes_ventilated_HIE.loc['AL000443', 'VG_on'] += vent_modes_ventilated_HIE.loc['AL000450']['VG_on']
vent_modes_ventilated_HIE.loc['AL000443', 'total'] += vent_modes_ventilated_HIE.loc['AL000450']['total']

In [ ]:
vent_modes_ventilated_HIE.loc[['AL000443', 'AL000450']]

In [ ]:
vent_modes_ventilated_HIE.drop('AL000450', inplace = True)
recordings.remove('AL000450')

In [ ]:
len(vent_modes_ventilated_HIE)

In [ ]:
vent_modes_ventilated_HIE = vent_modes_ventilated_HIE.reindex(recordings)

In [ ]:
len(vent_modes_ventilated_HIE)

In [ ]:
vent_modes_ventilated_HIE;

In [ ]:
len(recordings)

### Only keep the relevant ventilator and clinical data

In [ ]:
data_pars_measurements_ventilated_HIE = {rec: data_pars_measurements_ventilated_HIE[rec] for rec 
                                         in data_pars_measurements_ventilated_HIE if rec in recordings}

data_pars_settings_ventilated_HIE = {rec: data_pars_settings_ventilated_HIE[rec] for rec 
                                         in data_pars_settings_ventilated_HIE if rec in recordings}

data_pars_alarms_ventilated_HIE = {rec : data_pars_alarms_ventilated_HIE[rec] for rec 
                                         in data_pars_alarms_ventilated_HIE if rec in recordings}

vent_modes_ventilated_HIE = vent_modes_ventilated_HIE.reindex(recordings)

In [ ]:
len(data_pars_measurements_ventilated_HIE)

In [ ]:
clin_df_HIE = clin_df_HIE.loc[recordings]

In [ ]:
len(clin_df_HIE)

### Correct file type to float when required

In [ ]:
for recording in data_pars_measurements_ventilated_HIE:
    for column in data_pars_measurements_ventilated_HIE[recording].columns:
        try:
            data_pars_measurements_ventilated_HIE[recording][column] = \
                data_pars_measurements_ventilated_HIE[recording][column].astype('float')
        except:
            continue

In [ ]:
for recording in data_pars_settings_ventilated_HIE:
    for column in data_pars_settings_ventilated_HIE[recording].columns:
        try:
            data_pars_settings_ventilated_HIE[recording][column] = \
                data_pars_settings_ventilated_HIE[recording][column].astype('float')
        except:
            continue

### Statistics on clinical details of these cases

This considers all the __46 infants__ irrespective of the ventilator modes used

The _postnatal age_ shown in the table is the age at the end of the transfer

In [ ]:
len(clin_df_HIE)

In [ ]:
clin_df_HIE.info()

In [ ]:
clin_df_HIE_stats = round(clin_df_HIE.describe(percentiles = 
                        [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2)
clin_df_HIE_stats

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clinical_data_HIE.xlsx'))
clin_df_HIE.to_excel(writer, 'HIE_cases')
clin_df_HIE_stats.to_excel(writer, 'stats')
writer.save()

### Boxplots on selected clinical parameters 

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['recording duration']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot(clin_df_HIE['Duration'] / (60 * 1E+9), widths = 0.3,
        whis = 'range', showfliers = True,showmeans = True, meanprops = meanpointprops,
        medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylabel('minutes', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'HIE_recording_duration', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['Gestational age']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot(clin_df_HIE['Gestational Age (weeks)'] ,
            widths = 0.3, whis = 'range', showfliers = True, showmeans = True, medianprops=medianprops, 
            meanprops = meanpointprops,
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_ylim(35, 43)
ax.set_ylabel('weeks', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'HIE_gest_age', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['Birth weight']

medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot(clin_df_HIE['Birth Weight'], widths = 0.3, whis = 'range', showfliers = True,
             showmeans = True, meanprops = meanpointprops, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(2000, 5500)
ax.set_ylabel('grams', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'HIE_birth_weight', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

#### Postnatal age

The above postnatal age is calculated at the end of the transfer. Calculate the postnatal age at the start of the trimmed ventilator recordings

In [ ]:
postnatal_age_2 = {}

for recording in data_pars_measurements_ventilated_HIE:
    postnatal_age_2[recording] = data_pars_measurements_ventilated[recording].index[0] - \
                                 clin_df_HIE.loc[recording]['Date of Birth']

Postnatal_age_2 = pd.DataFrame.from_dict(postnatal_age_2, orient = 'index', columns = ['Postnatal_age_2'])

clin_df_HIE = pd.concat([clin_df_HIE, Postnatal_age_2], axis = 1)

In [ ]:
clin_df_HIE.head()

In [ ]:
clin_df_HIE['Postnatal_age_2'].describe()

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['Postnatal age']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot(clin_df_HIE['Postnatal_age_2'] / 3600000000000,
            widths = 0.3, whis = 'range', showfliers = True, showmeans = True, medianprops=medianprops, 
            meanprops = meanpointprops,
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

#ax.set_ylim(35, 43)
ax.set_ylabel('hours', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'HIE_postnatal_age', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

### Import file with manually collected additional clinical data

In [ ]:
clin_df_HIE_2 = pd.read_excel('%s/%s' % (CWD, 'Clinical data_HIE_Lajos.xlsx'))
clin_df_HIE_2.set_index('Recording_ID', inplace = True)
# Limit dataset to the included recordings only
clin_df_HIE_2 = clin_df_HIE_2.reindex(clin_df_HIE.index)
len(clin_df_HIE_2)

In [ ]:
len(clin_df_HIE)

In [ ]:
clin_df_HIE_2.columns

In [ ]:
to_keep = ['Diagnosis', 'Mode of delivery', 'Apgar (1 / 5 / 10)', 'Apgar_5', 'Apgar_10',
       'Seizure', 'Cooling (active / passive)', 'Temp on arrival (C ) to referral unit', 'Temp on handover (C )',
       'Anticonvulsant', 'Sedatives', 'Muscle relaxant', 'Inotropes','Curosurf']

clin_df_HIE_2 = clin_df_HIE_2[to_keep]

In [ ]:
clin_df_HIE_2.head()

In [ ]:
clin_df_HIE_2.info()

### Statistics on additional clinical details

This considers all the __46 infants__ irrespective of the ventilator modes used

##### Mode of delivery

In [ ]:
clin_df_HIE_2['Mode of delivery'].value_counts(dropna = False)

- SVD: 21
- Instrumental delivery: 2
- ElCS: 1
- EmCS: 21
- n/a: 1

##### Active of passive cooling and temperatures on arrical and at handover

In [ ]:
clin_df_HIE_2['Cooling (active / passive)'].value_counts(dropna = False)

- Active: 35
- Passive: 10
- No: 1

Manual lookup: The baby who had no cooling was still cooling down during transport (checked manually) so count it as passive

In [ ]:
clin_df_HIE_2['Temp on arrival (C ) to referral unit'].describe()

In [ ]:
clin_df_HIE_2['Temp on handover (C )'].describe()

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['At arrival', 'At handover']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df_HIE_2['Temp on arrival (C ) to referral unit'].dropna(), 
             clin_df_HIE_2['Temp on handover (C )'].dropna()], widths = 0.5,
        whis = 'range', showfliers = True,showmeans = True, meanprops = meanpointprops,
        medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(27, 40)
ax.set_ylabel('Degree of Celsius', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'HIE_temperatures', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

##### Presence of seizures and use of anticonvulsants

In [ ]:
clin_df_HIE_2['Seizure'].value_counts(dropna = False)

- Seizure: 11
- No seizure: 33
- n/a: 2

In [ ]:
clin_df_HIE_2['Anticonvulsant'].value_counts(dropna = False)

- Phenobarbital: 11
- No: 35

##### Sedative and muscle relaxant medication

In [ ]:
clin_df_HIE_2['Sedatives'].value_counts(dropna = False)

- Fentanyl alone: 14
- Midazolam alone: 2
- Thiopental alone: 1
- Morphine alone: 1
- Fentanyl + Thiopental: 13
- Fentanyl + Midazolam: 2
- No: 13

In [ ]:
clin_df_HIE_2['Muscle relaxant'].value_counts(dropna = False)

- Rocuronium: 6
- Atracurium: 1
- No: 39

##### Other medications

In [ ]:
clin_df_HIE_2['Inotropes'].value_counts(dropna = False)

- Dopamine alone: 5
- Dobutamine alone: 1
- Dopamine + Dobutamine: 1
- Dopamine + Noradrenaline + Hydrocortison: 1
- No: 38

In [ ]:
clin_df_HIE_2['Curosurf'].value_counts(dropna = False)

- Curosurf: 4
- No: 41
- n/a: 1

### How many cases of this had predominantly SIMV and VG ventilation (>90% of the recording)

There are no IPPV or PSV recordings

In [ ]:
vent_modes_ventilated_HIE.sum()

In [ ]:
vent_modes_ventilated_HIE.drop(['IPPV', 'PSV'], axis = 1, inplace = True)

In [ ]:
vent_modes_ventilated_HIE;

###### SIMV_VG and SIMV_noVG

In [ ]:
SIMV  = vent_modes_ventilated_HIE[vent_modes_ventilated_HIE['SIMV'] >= 0.9 * vent_modes_ventilated_HIE['total']]
len(SIMV)

In [ ]:
VG = vent_modes_ventilated_HIE[vent_modes_ventilated_HIE['VG_on'] >= 0.9 * vent_modes_ventilated_HIE['total']]
len(VG)

In [ ]:
no_VG = vent_modes_ventilated_HIE[vent_modes_ventilated_HIE['VG_on'] <= 0.1 * vent_modes_ventilated_HIE['total']]
len(no_VG)

In [ ]:
SIMV_VG = vent_modes_ventilated_HIE[(vent_modes_ventilated_HIE['SIMV'] >= 0.9 * vent_modes_ventilated_HIE['total'])
                                & (vent_modes_ventilated_HIE['VG_on'] >= 0.9 * vent_modes_ventilated_HIE['total'])]

In [ ]:
len(SIMV_VG)

In [ ]:
SIMV_VG;

In [ ]:
simv_vg = sorted(SIMV_VG.index)

In [ ]:
SIMV_noVG = vent_modes_ventilated_HIE[(vent_modes_ventilated_HIE['SIMV'] >= 0.9 * vent_modes_ventilated_HIE['total'])
                                & (vent_modes_ventilated_HIE['VG_on'] <= 0.1 * vent_modes_ventilated_HIE['total'])]

In [ ]:
len(SIMV_noVG)

In [ ]:
SIMV_noVG;

In [ ]:
simv_novg = sorted(SIMV_noVG.index)

##### SIPPV (with or without VG)

In [ ]:
SIPPV  = vent_modes_ventilated_HIE[vent_modes_ventilated_HIE['SIPPV'] >= 0.1 * vent_modes_ventilated_HIE['total']]
len(SIPPV)

In [ ]:
SIPPV

In [ ]:
SIPPV['SIPPV'] / SIPPV['total']

In [ ]:
SIPPV['SIMV'] / SIPPV['total']

- 2 babies had predominantly (>90%) SIPPV, 1 with VG 1 without VG
- 3 babies received significant amount (33-66%) of both SIPPV-VG and some SIMV-VG 
- 1 baby received some (<20%) SIPPV-VG but otherwise SIMV-VG)

In [ ]:
sippv = sorted(SIPPV.index)

##### SIMV-PS (with or without VG)

In [ ]:
SIMVPSV  = vent_modes_ventilated_HIE[vent_modes_ventilated_HIE['SIMVPSV'] >= 0.1 * vent_modes_ventilated_HIE['total']]
len(SIMVPSV)

Two babies received SIMV-VG with pressure support

In [ ]:
SIMVPSV

In [ ]:
simvpsv = sorted(SIMVPSV.index)

### Limit ventilator data to SIMV-VG and SIMV-noVG 

In [ ]:
data_pars_measurements_simv_vg = {}
data_pars_settings_simv_vg = {}
data_pars_alarms_simv_vg = {}

for recording in simv_vg:
    mask = ((data_pars_settings_ventilated_HIE[recording]['Ventilator_mode'] == 'SIMV') & 
        (data_pars_settings_ventilated_HIE[recording]['VG_state'] == 'on'))
    
    data_pars_measurements_simv_vg[recording] = data_pars_measurements_ventilated_HIE[recording][mask]
    data_pars_settings_simv_vg[recording] = data_pars_settings_ventilated_HIE[recording][mask]
    data_pars_alarms_simv_vg[recording] = data_pars_alarms_ventilated_HIE[recording][mask]

In [ ]:
# Duration of recordings in seconds is twice these numbers as 0.5 Hz sampling rate
for recording in simv_vg:
    print(recording, len(data_pars_measurements_simv_vg[recording]), 
          len(data_pars_measurements_ventilated_HIE[recording]))

In [ ]:
data_pars_measurements_simv_novg = {}
data_pars_settings_simv_novg = {}
data_pars_alarms_simv_novg = {}

for recording in simv_novg:
    if 'VG_state' in data_pars_settings_ventilated_HIE[recording].columns:
        mask = ((data_pars_settings_ventilated_HIE[recording]['Ventilator_mode'] == 'SIMV') & 
            (data_pars_settings_ventilated_HIE[recording]['VG_state'] != 'on'))
    
        data_pars_measurements_simv_novg[recording] = data_pars_measurements_ventilated_HIE[recording][mask]
        data_pars_settings_simv_novg[recording] = data_pars_settings_ventilated_HIE[recording][mask]
        data_pars_alarms_simv_novg[recording] = data_pars_alarms_ventilated_HIE[recording][mask]
                                                       
    else:
        
        data_pars_measurements_simv_novg[recording] = data_pars_measurements_ventilated_HIE[recording]
        data_pars_settings_simv_novg[recording] = data_pars_settings_ventilated_HIE[recording]
        data_pars_alarms_simv_novg[recording] = data_pars_alarms_ventilated_HIE[recording]                                              
                                                           

In [ ]:
# Duration of recordings in seconds is twice these numbers as 0.5 Hz sampling rate
for recording in simv_novg:
    print(recording, len(data_pars_measurements_simv_novg[recording]), 
          len(data_pars_measurements_ventilated_HIE[recording]))

### Add MVspon to the recordings

In [ ]:
for recording in simv_vg:
    data_pars_measurements_simv_vg[recording].loc[:, 'MVspon'] = \
        100 - data_pars_measurements_simv_vg[recording]['MVresp']

In [ ]:
for recording in simv_novg:
    data_pars_measurements_simv_novg[recording].loc[:, 'MVspon'] = \
        100 - data_pars_measurements_simv_novg[recording]['MVresp']

### Add Pdiff and VTdiff to the recordings

VTdiff = VTemand_resp_kg - VTset ("VGset" in VG recordings)

Pdiff = Pmax - PIP ("PIP_set" in VG recordings)

In [ ]:
for recording in simv_vg:
    data_pars_measurements_simv_vg[recording].loc[:, 'VTdiff'] = \
        data_pars_measurements_ventilated_HIE[recording]['VTemand_resp_kg'] - \
        data_pars_settings_ventilated_HIE[recording]['VG_set_kg']

In [ ]:
for recording in simv_vg:
    data_pars_measurements_simv_vg[recording].loc[:, 'Pdiff'] = \
        data_pars_settings_ventilated_HIE[recording]['PIP_set'] - \
        data_pars_measurements_ventilated_HIE[recording]['PIP']

### Recording durations

In [ ]:
duration_simv_vg = []
duration_simv_novg = []

for recording in simv_vg:
    # In seconds. The multiplication is needed as it is 0.5 Hz sampling data
    duration_simv_vg.append(len(data_pars_measurements_simv_vg[recording]) * 2)

duration_simv_vg = Series(duration_simv_vg)
       
for recording in simv_novg:
    # In seconds. The multiplication is needed as it is 0.5 Hz sampling data
    duration_simv_novg.append(len(data_pars_measurements_simv_novg[recording]) * 2)

duration_simv_novg = Series(duration_simv_novg)

In [ ]:
duration_simv_vg.describe() / 60

In [ ]:
duration_simv_novg.describe() / 60

In [ ]:
stats.mannwhitneyu(duration_simv_vg, duration_simv_novg)

In [ ]:
# Total duration of recordings
duration_simv_vg.sum(), duration_simv_vg.sum() / 60, duration_simv_vg.sum() / 3600

In [ ]:
# Total duration of recordings
duration_simv_novg.sum(), duration_simv_novg.sum() / 60, duration_simv_novg.sum() / 3600

In [ ]:
(duration_simv_vg.sum() + duration_simv_novg.sum()) / 3600

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV-VG', 'SIMV' ]

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([duration_simv_vg / 60 , duration_simv_novg / 60], widths = 0.5, 
        whis = 'range', showfliers = True,showmeans = True, meanprops = meanpointprops, 
        medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_ylim(0, 180)
ax.set_xlabel('', size = 14)
ax.set_ylabel('Recording duration (minutes)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
ax.set_title('')
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'HIE_recording_duration_SIMV_VG_no_VG', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

### Comparative boxplots on clinical details between SIMV-VG and no SIMV-noVG

28 SIMV-VG recordings, 8 SIMV recordings

In [ ]:
clin_df_VG = clin_df_HIE.loc[simv_vg]
clin_df_no_VG = clin_df_HIE.loc[simv_novg]

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clinical_data_HIE_VG_noVG.xlsx'))
clin_df_VG.to_excel(writer, 'clin_df_VG')
clin_df_no_VG.to_excel(writer, 'clin_df_no_VG')
writer.save()

### Gestational age

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV-VG', 'SIMV' ]

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df_VG['Gestational Age (weeks)'] , 
             clin_df_no_VG['Gestational Age (weeks)'] ], widths = 0.5, meanprops = meanpointprops, 
        whis = 'range', showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_ylim(33, 45)
ax.set_xlabel('', size = 14)
ax.set_ylabel('Gestational age (weeks)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
ax.set_title('Gestational age')
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'HIE_gest_age_SIMV_VG_no_VG', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

### Postnatal Age

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV-VG', 'SIMV' ]

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df_VG['Postnatal_age_2']   / 3600000000000,
             clin_df_no_VG['Postnatal_age_2'] / 3600000000000],  
             widths = 0.5, meanprops = meanpointprops, 
            whis = 'range', showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
            whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_ylim(0, 24)
ax.set_xlabel('', size = 14)
ax.set_ylabel('Postnatal age (hours)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
ax.set_title('Gestational age')
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'HIE_postnatal_age_SIMV_VG_no_VG', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

### Birth weight

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV-VG', 'SIMV' ]

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df_VG['Birth Weight'], 
             clin_df_no_VG['Birth Weight']], widths = 0.5, meanprops = meanpointprops, 
        whis = 'range', showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)
ax.set_ylim(2000, 5400)
ax.set_xlabel('', size = 14)
ax.set_ylabel('Birth weight (grams)', size = 14)
ax.set_title('')
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'HIE_birth_weight_SIMV_VG_no_VG', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

### Comparative statistics between gestational age and birth weight, SIMV-VG versus SIMV-noVG

In [ ]:
pars = ['Gestational Age (weeks)', 'Birth Weight',]

clin_df_stats_VG = round(DataFrame([clin_df_VG[pars].mean(), 
                                    clin_df_VG[pars].std()]).T, 2)
clin_df_stats_VG.columns = ['mean', 'SD']

clin_df_stats_no_VG = round(DataFrame([clin_df_no_VG[pars].mean(), 
                                       clin_df_no_VG[pars].std()]).T, 2)
clin_df_stats_no_VG.columns = ['mean', 'SD']

clin_df_stats_VG_no_VG = pd.concat([clin_df_stats_VG, clin_df_stats_no_VG], axis = 1,
                              keys = ['VG', 'no VG'])

In [ ]:
ttest_stat = []
ttest_p = []
for par in pars:
    ttest_stat.append(stats.ttest_ind(clin_df_VG[par], clin_df_no_VG[par])[0])
    ttest_p.append(stats.ttest_ind(clin_df_VG[par],  clin_df_no_VG[par])[1])

clin_df_stats_VG_no_VG['p'] = ttest_p  

In [ ]:
clin_df_stats_VG_no_VG

In [ ]:
# For postnatal age

stats.ttest_ind(clin_df_VG['Postnatal_age_2'].apply(lambda x: x.total_seconds()), 
                clin_df_no_VG['Postnatal_age_2'].apply(lambda x: x.total_seconds()))

In [ ]:
(clin_df_VG['Postnatal_age_2'].apply(lambda x: x.total_seconds()).mean() / 3600,
 clin_df_VG['Postnatal_age_2'].apply(lambda x: x.total_seconds()).std() / 3600)
 

In [ ]:
(clin_df_no_VG['Postnatal_age_2'].apply(lambda x: x.total_seconds()).mean() / 3600,
clin_df_no_VG['Postnatal_age_2'].apply(lambda x: x.total_seconds()).std() / 3600)

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clin_df_stats_SIMV_VG_no_VG.xlsx'))
clin_df_stats_VG_no_VG.to_excel(writer, 'VG_no_VG')
writer.save()

### Calculate and compare the respiratory severity score (RSS) for SIMV-VG and SIMV-noVG recordings

In [ ]:
# 5 minutes = 150 data points with 0.5 Hz sampling rate

rss_simv_vg = {}
for recording in simv_vg:
    rss_simv_vg[recording] = round((data_pars_settings_simv_vg[recording]['FiO2_set'] * 
                              data_pars_measurements_simv_vg[recording]['MAP'] / 100).dropna().mean(), 2)
    
rss_simv_novg = {}
for recording in simv_novg:
    rss_simv_novg[recording] = round((data_pars_settings_simv_novg[recording]['FiO2_set'] * 
                              data_pars_measurements_simv_novg[recording]['MAP'] / 100).dropna().mean(), 2)
    
rss_simv_vg_frame = DataFrame([rss_simv_vg]).T
rss_simv_vg_frame.columns = ['respiratory severity score']

rss_simv_novg_frame = DataFrame([rss_simv_novg]).T
rss_simv_novg_frame.columns = ['respiratory severity score']

In [ ]:
rss_simv_vg_frame;

In [ ]:
rss_simv_novg_frame;

In [ ]:
round(rss_simv_vg_frame.describe(), 1)

In [ ]:
round(rss_simv_novg_frame.describe(), 1)

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV-VG', 'SIMV' ]

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([rss_simv_vg_frame['respiratory severity score'], 
             rss_simv_novg_frame['respiratory severity score']], widths = 0.5, meanprops = meanpointprops, 
        whis = 'range', showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)
#ax.set_ylim(2000, 5400)
ax.set_xlabel('', size = 14)
ax.set_ylabel('Respiratory severity score', size = 14)
ax.set_title('')
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'HIE_RSS_SIMV_VG_no_VG', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

In [ ]:
stats.mannwhitneyu(rss_simv_vg_frame, rss_simv_novg_frame)

### Clinical problems

In [ ]:
clin_df_VG;

In [ ]:
for index, *item in clin_df_HIE.loc[simv_vg][['Pathology_English']].itertuples():
    print(index, item, '\n')

In [ ]:
clin_df_no_VG;

In [ ]:
for index, *item in clin_df_HIE.loc[simv_novg][['Pathology_English']].itertuples():
    print(index, item, '\n')

### Temperature on arrival and at handover

##### At arrival on the referring Unit

In [ ]:
clin_df_HIE_2.loc[simv_vg]['Temp on arrival (C ) to referral unit'].describe()

In [ ]:
clin_df_HIE_2.loc[simv_novg]['Temp on arrival (C ) to referral unit'].describe()

In [ ]:
stats.mannwhitneyu(clin_df_HIE_2.loc[simv_vg]['Temp on arrival (C ) to referral unit'], 
                   clin_df_HIE_2.loc[simv_novg]['Temp on arrival (C ) to referral unit'])

In [ ]:
clin_df_HIE_2.loc[simv_vg]['Temp on handover (C )'].describe()

In [ ]:
clin_df_HIE_2.loc[simv_novg]['Temp on handover (C )'].describe()

In [ ]:
stats.mannwhitneyu(clin_df_HIE_2.loc[simv_vg]['Temp on handover (C )'], 
                   clin_df_HIE_2.loc[simv_novg]['Temp on handover (C )'])

### Other clinical characteristics

#### Mode of delivery

In [ ]:
clin_df_HIE_2.loc[simv_vg]['Mode of delivery'].value_counts(dropna = False)

- SVD: 15
- Instrumental delivery: 0
- ElCS: 1
- EmCS: 11
- n/a: 1

In [ ]:
clin_df_HIE_2.loc[simv_novg]['Mode of delivery'].value_counts()

- SVD: 1
- Instrumental delivery: 1
- ElCS: 1
- EmCS: 6

##### 5-min Apgar score

In [ ]:
clin_df_HIE_2.loc[simv_vg]['Apgar_5'].describe()

In [ ]:
clin_df_HIE_2.loc[simv_novg]['Apgar_5'].describe()

In [ ]:
stats.mannwhitneyu(clin_df_HIE_2.loc[simv_vg]['Apgar_5'], clin_df_HIE_2.loc[simv_novg]['Apgar_5'])

##### 10-min Apgar score

In [ ]:
clin_df_HIE_2.loc[simv_vg]['Apgar_10'].describe()

In [ ]:
clin_df_HIE_2.loc[simv_novg]['Apgar_10'].describe()

In [ ]:
stats.mannwhitneyu(clin_df_HIE_2.loc[simv_vg]['Apgar_10'], clin_df_HIE_2.loc[simv_novg]['Apgar_10'])

##### Active of passive cooling

The ones who had no cooling was still cooling down during transport so count it as passive

In [ ]:
clin_df_HIE_2.loc[simv_vg]['Cooling (active / passive)'].value_counts(dropna = False)

- Active cooling: 22
- Passive cooling: 5
- No cooling: 1

Manual lookup: The baby who had no cooling was still cooling down during transport (checked manually) so count it as passive

In [ ]:
clin_df_HIE_2.loc[simv_novg]['Cooling (active / passive)'].value_counts(dropna = False)

- Active cooling: 6
- Passive cooling: 2

In [ ]:
active_passive_oddsratio, active_passive_pvalue = stats.fisher_exact([[22, 6], [6, 2]])
active_passive_oddsratio, active_passive_pvalue

##### Seizures

In [ ]:
clin_df_HIE_2.loc[simv_vg]['Seizure'].value_counts(dropna = False)

- Seizure: 9
- No seizure: 18
- n/a: 1

In [ ]:
clin_df_HIE_2.loc[simv_novg]['Seizure'].value_counts(dropna = False)

- Seizure: 1
- No seizure: 7


### Medications

##### Sedation

In [ ]:
clin_df_HIE_2.loc[simv_vg]['Sedatives'].value_counts(dropna = False)

- Fentanyl alone: 9
- Morphine alone: 1
- Midazolam alone: 0
- Thiopental alone: 1
- Fentanyl + Thiopental: 9
- Fentanyl + Midazolam: 1
- No: 7

In [ ]:
clin_df_HIE_2.loc[simv_novg]['Sedatives'].value_counts(dropna = False)

- Fentanyl alone: 2
- Morphine alone: 0
- Midazolam alone: 1
- Thiopental alone: 0
- Fentanyl + Thiopental: 2
- Fentanyl + Midazolam: 1
- No: 2

In [ ]:
sedation_no_sedation_oddsratio, sedation_no_sedation_pvalue = stats.fisher_exact([[20, 8], [6, 2]])
sedation_no_sedation_oddsratio, sedation_no_sedation_pvalue

##### Muscle relaxant

In [ ]:
clin_df_HIE_2.loc[simv_vg]['Muscle relaxant'].value_counts()

- Rocuronium: 3
- Atracurium: 0
- No: 25

In [ ]:
clin_df_HIE_2.loc[simv_novg]['Muscle relaxant'].value_counts()

- Rocuronium: 2
- Atracurium: 1
- No: 5

In [ ]:
relaxant_oddsratio, relaxant_pvalue = stats.fisher_exact([[24, 4], [5, 3]])
relaxant_oddsratio, relaxant_pvalue

##### Anticonvulsant

In [ ]:
clin_df_HIE_2.loc[simv_vg]['Anticonvulsant'].value_counts()

- Phenobarbital: 9
- No: 19

In [ ]:
clin_df_HIE_2.loc[simv_novg]['Anticonvulsant'].value_counts()

- Phenobarbital: 1
- No: 7

In [ ]:
anticonvulsant_oddsratio, anticonvulsant_pvalue = stats.fisher_exact([[19, 9], [7, 1]])
anticonvulsant_oddsratio, anticonvulsant_pvalue

In [ ]:
# If == 2 they were not on any sedative
clin_df_HIE_2.loc[simv_vg][['Sedatives', 'Anticonvulsant',]]

In [ ]:
# If == 2 they were not on any sedative
((clin_df_HIE_2.loc[simv_vg][['Sedatives', 'Anticonvulsant',]] == 'no').sum(axis = 1) == 2).sum()

In [ ]:
# If == 2 they were not on any sedative
clin_df_HIE_2.loc[simv_novg][['Sedatives', 'Anticonvulsant',]]

In [ ]:
# If == 2 they were not on any sedative
(clin_df_HIE_2.loc[simv_novg][['Sedatives', 'Anticonvulsant',]] == 'no').sum(axis = 1)

##### Inotropic support

In [ ]:
clin_df_HIE_2.loc[simv_vg]['Inotropes'].value_counts()

- None: 24
- Dopamine: 2
- Dobutamine: 1
- Dopamine, Noradrenaline, Hydrocortison: 1

In [ ]:
clin_df_HIE_2.loc[simv_novg]['Inotropes'].value_counts()

- None: 6
- Dopamine: 1
- Dopamine, Dobutamine: 1

##### Surfactant 

In [ ]:
clin_df_HIE_2.loc[simv_vg]['Curosurf'].value_counts()

- No: 25
- Yes: 3

In [ ]:
clin_df_HIE_2.loc[simv_novg]['Curosurf'].value_counts()

- No: 26
- Yes: 1

### Calculate and compare ventilator parameters

In [ ]:
data_pars_settings_ventilated_HIE['AL000008'].info()

In [ ]:
data_pars_settings_simv_vg['AL000008'].info()

##### SIMV-VG

In [ ]:
stats_measurements_simv_vg = {} 

for recording in simv_vg:
    stats_measurements_simv_vg[recording] = \
        round(data_pars_measurements_simv_vg[recording].describe(percentiles = 
                                                    (0.05, 0.25, 0.5, 0.75, 0.95)), 2)
    stats_measurements_simv_vg[recording].index = ['data_points', 'mean', 'SD', 'min', '5pc', 
                                                      '25pc', 'median', '75pc', '95pc', 'max']

In [ ]:
stats_settings_simv_vg = {} 

for recording in simv_vg:
    stats_settings_simv_vg[recording] = \
        round(data_pars_settings_simv_vg[recording].describe(percentiles = 
                                                    (0.05, 0.25, 0.5, 0.75, 0.95)), 2)
    stats_settings_simv_vg[recording].index = ['data_points', 'mean', 'SD', 'min', '5pc', 
                                                      '25pc', 'median', '75pc', '95pc', 'max']

##### SIMV-noVG

In [ ]:
stats_measurements_simv_novg = {} 

for recording in simv_novg:
    stats_measurements_simv_novg[recording] = \
        round(data_pars_measurements_simv_novg[recording].describe(percentiles = 
                                                    (0.05, 0.25, 0.5, 0.75, 0.95)), 2)
    stats_measurements_simv_novg[recording].index = ['data_points', 'mean', 'SD', 'min', '5pc', 
                                                      '25pc', 'median', '75pc', '95pc', 'max']

In [ ]:
stats_settings_simv_novg = {} 

for recording in simv_novg:
    stats_settings_simv_novg[recording] = \
        round(data_pars_settings_simv_novg[recording].describe(percentiles = 
                                                    (0.05, 0.25, 0.5, 0.75, 0.95)), 2)
    stats_settings_simv_novg[recording].index = ['data_points', 'mean', 'SD', 'min', '5pc', 
                                                      '25pc', 'median', '75pc', '95pc', 'max']

## Create table with statistics for all cases and all relevant parameters

### Measured ventilator parameters

In [ ]:
stats_measurements_simv_vg_all = pd.concat(stats_measurements_simv_vg, axis = 1).T
# Remove measured parameters not given in case of mechanical ventilation
stats_measurements_simv_vg_all.dropna(how = 'all', subset = ['mean', 'SD', 'min', '5pc', 
                                '25pc', 'median', '75pc', '95pc', 'max'], axis = 0, inplace = True)

stats_measurements_simv_vg_all_2 = \
    stats_measurements_simv_vg_all.unstack()[['mean', 'SD']].swaplevel(axis = 1).sort_index(level = 0, axis = 1, ascending = False)
stats_measurements_simv_vg_all_3 = \
    stats_measurements_simv_vg_all.unstack()[['median', '25pc', '75pc']].swaplevel(axis = 1).sort_index(level = 0, axis = 1, ascending = False)

In [ ]:
stats_measurements_simv_novg_all = pd.concat(stats_measurements_simv_novg, axis = 1).T
# Remove measured parameters not given in case of mechanical ventilation
stats_measurements_simv_novg_all.dropna(how = 'all', subset = ['mean', 'SD', 'min', '5pc', 
                                '25pc', 'median', '75pc', '95pc', 'max'], axis = 0, inplace = True)
stats_measurements_simv_novg_all_2 = \
    stats_measurements_simv_novg_all.unstack()[['mean', 'SD']].swaplevel(axis = 1).sort_index(level = 0, axis = 1, ascending = False)
stats_measurements_simv_novg_all_3 = \
    stats_measurements_simv_novg_all.unstack()[['median', '25pc', '75pc']].swaplevel(axis = 1).sort_index(level = 0, axis = 1, ascending = False)

In [ ]:
stats_measurements_simv_vg_all.loc['AL000008']

In [ ]:
stats_measurements_simv_vg_all_2.head()

In [ ]:
stats_measurements_simv_vg_all_3.head()

### Ventilator settings

In [ ]:
stats_settings_simv_vg_all = pd.concat(stats_settings_simv_vg, axis = 1).T
# Remove set parameters not given in case of mechanical ventilation
stats_settings_simv_vg_all.dropna(how = 'all', subset = ['mean', 'SD', 'min', '5pc', 
                                '25pc', 'median', '75pc', '95pc', 'max'], axis = 0, inplace = True)
stats_settings_simv_vg_all_2 = \
    stats_settings_simv_vg_all.unstack()[['mean', 'SD']].swaplevel(axis = 1).sort_index(level = 0, axis = 1, ascending = False)
stats_settings_simv_vg_all_3 = \
    stats_settings_simv_vg_all.unstack()[['median', '25pc', '75pc']].swaplevel(axis = 1).sort_index(level = 0, axis = 1, ascending = False)


In [ ]:
stats_settings_simv_novg_all = pd.concat(stats_settings_simv_novg, axis = 1).T
# Remove set parameters not given in case of mechanical ventilation
stats_settings_simv_novg_all.dropna(how = 'all', subset = ['mean', 'SD', 'min', '5pc', 
                                '25pc', 'median', '75pc', '95pc', 'max'], axis = 0, inplace = True)
stats_settings_simv_novg_all_2 = \
    stats_settings_simv_novg_all.unstack()[['mean', 'SD']].swaplevel(axis = 1).sort_index(level = 0, axis = 1, ascending = False)
stats_settings_simv_novg_all_3 = \
    stats_settings_simv_novg_all.unstack()[['median', '25pc', '75pc']].swaplevel(axis = 1).sort_index(level = 0, axis = 1, ascending = False)

In [ ]:
stats_settings_simv_novg_all.loc['AL000019']

In [ ]:
stats_settings_simv_vg_all_2.head()

In [ ]:
stats_settings_simv_vg_all_3.head()

### Select the subset to be presented with the appropriate aggregate statistics

### VG recordings

In [ ]:
data_pars_measurements_simv_vg['AL000008'].info()

In [ ]:
# For parametrically distributed data use mean and SD

columns_to_keep_1 = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTemand_resp_kg', 'VTdiff',]

stats_measurements_simv_vg_all_filt_1 = \
    stats_measurements_simv_vg_all.unstack()[['mean', 'SD']].swaplevel(axis = 1)
stats_measurements_simv_vg_all_filt_1 = \
    stats_measurements_simv_vg_all_filt_1.sort_index(axis = 1, level = 0, ascending = False)
stats_measurements_simv_vg_all_filt_1 = stats_measurements_simv_vg_all_filt_1[columns_to_keep_1]
stats_measurements_simv_vg_all_filt_1.head()

In [ ]:
# For non_parametrically distributed data use median, 25pc, 75pc

columns_to_keep_2 = ['MVresp', 'MVspon', 'Pdiff']

stats_measurements_simv_vg_all_filt_2 = \
    stats_measurements_simv_vg_all.unstack()[['median', '25pc', '75pc']].swaplevel(axis = 1)
stats_measurements_simv_vg_all_filt_2 = \
    stats_measurements_simv_vg_all_filt_2.sort_index(axis = 1, level = 0, ascending = False)
stats_measurements_simv_vg_all_filt_2 = stats_measurements_simv_vg_all_filt_2[columns_to_keep_2]
stats_measurements_simv_vg_all_filt_2.head()

In [ ]:
# For parametrically distributed data use mean and SD

columns_to_keep_3 = ['RR_set', 'VG_set_kg']

stats_settings_simv_vg_all_filt_3 = \
    stats_settings_simv_vg_all.unstack()[['mean', 'SD']].swaplevel(axis = 1)
stats_settings_simv_vg_all_filt_3 = \
    stats_settings_simv_vg_all_filt_3.sort_index(axis = 1, level = 0, ascending = False)
stats_settings_simv_vg_all_filt_3 = stats_settings_simv_vg_all_filt_3[columns_to_keep_3]
stats_settings_simv_vg_all_filt_3.head()

In [ ]:
# For non-parametrically distributed data use median and 25pc, 75pc
columns_to_keep_4 = ['FiO2_set']

stats_settings_simv_vg_all_filt_4 = \
    stats_settings_simv_vg_all.unstack()[['median', '25pc', '75pc']].swaplevel(axis = 1)
stats_settings_simv_vg_all_filt_4 = \
    stats_settings_simv_vg_all_filt_4.sort_index(axis = 1, level = 0, ascending = False)
stats_settings_simv_vg_all_filt_4 = stats_settings_simv_vg_all_filt_4[columns_to_keep_4]
stats_settings_simv_vg_all_filt_4.head()

In [ ]:
stats_simv_vg_selected = pd.concat([stats_measurements_simv_vg_all_filt_1, stats_measurements_simv_vg_all_filt_2,
                                    stats_settings_simv_vg_all_filt_3, stats_settings_simv_vg_all_filt_4],
                                   axis = 1)

In [ ]:
stats_simv_vg_selected.head()

### Non-VG recordings

In [ ]:
# For parametrically distributed data use mean and SD

columns_to_keep_1 = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTemand_resp_kg',]

stats_measurements_simv_novg_all_filt_1 = \
    stats_measurements_simv_novg_all.unstack()[['mean', 'SD']].swaplevel(axis = 1)
stats_measurements_simv_novg_all_filt_1 = \
    stats_measurements_simv_novg_all_filt_1.sort_index(axis = 1, level = 0, ascending = False)
stats_measurements_simv_novg_all_filt_1 = stats_measurements_simv_novg_all_filt_1[columns_to_keep_1]
stats_measurements_simv_novg_all_filt_1.head()

In [ ]:
# For non_parametrically distributed data use median, 25pc, 75pc

columns_to_keep_2 = ['MVresp', 'MVspon',]

stats_measurements_simv_novg_all_filt_2 = \
    stats_measurements_simv_novg_all.unstack()[['median', '25pc', '75pc']].swaplevel(axis = 1)
stats_measurements_simv_novg_all_filt_2 = \
    stats_measurements_simv_novg_all_filt_2.sort_index(axis = 1, level = 0, ascending = False)
stats_measurements_simv_novg_all_filt_2 = stats_measurements_simv_novg_all_filt_2[columns_to_keep_2]
stats_measurements_simv_novg_all_filt_2.head()

In [ ]:
# For parametrically distributed data use mean and SD

columns_to_keep_3 = ['RR_set',]

stats_settings_simv_novg_all_filt_3 = \
    stats_settings_simv_novg_all.unstack()[['mean', 'SD']].swaplevel(axis = 1)
stats_settings_simv_novg_all_filt_3 = \
    stats_settings_simv_novg_all_filt_3.sort_index(axis = 1, level = 0, ascending = False)
stats_settings_simv_novg_all_filt_3 = stats_settings_simv_novg_all_filt_3[columns_to_keep_3]
stats_settings_simv_novg_all_filt_3.head()

In [ ]:
# For non-parametrically distributed data use median and 25pc, 75pc
columns_to_keep_4 = ['FiO2_set']

stats_settings_simv_novg_all_filt_4 = \
    stats_settings_simv_novg_all.unstack()[['median', '25pc', '75pc']].swaplevel(axis = 1)
stats_settings_simv_novg_all_filt_4 = \
    stats_settings_simv_novg_all_filt_4.sort_index(axis = 1, level = 0, ascending = False)
stats_settings_simv_novg_all_filt_4 = stats_settings_simv_novg_all_filt_4[columns_to_keep_4]
stats_settings_simv_novg_all_filt_4.head()

In [ ]:
stats_simv_novg_selected = pd.concat([stats_measurements_simv_novg_all_filt_1, stats_measurements_simv_novg_all_filt_2,
                                    stats_settings_simv_novg_all_filt_3, stats_settings_simv_novg_all_filt_4],
                                   axis = 1)

In [ ]:
stats_simv_novg_selected.head()

### Export statistics to a multisheet Excel file and pickle archive

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_measurements_simv_vg.xlsx'))
for recording in simv_vg:
    stats_measurements_simv_vg[recording].to_excel(writer, recording)
writer.save()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_measurements_simv_novg.xlsx'))
for recording in simv_novg:
    stats_measurements_simv_novg[recording].to_excel(writer, recording)
writer.save()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_settings_simv_vg.xlsx'))
for recording in simv_vg:
    stats_settings_simv_vg[recording].to_excel(writer, recording)
writer.save()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_settings_simv_novg.xlsx'))
for recording in simv_novg:
    stats_settings_simv_novg[recording].to_excel(writer, recording)
writer.save()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_simv_vg_all.xlsx'))
stats_measurements_simv_vg_all.to_excel(writer, 'measurements')
stats_measurements_simv_vg_all_2.to_excel(writer, 'measurements_2')
stats_measurements_simv_vg_all_3.to_excel(writer, 'measurements_3')
stats_settings_simv_vg_all.to_excel(writer, 'settings')
stats_settings_simv_vg_all_2.to_excel(writer, 'settings_2')
stats_settings_simv_vg_all_3.to_excel(writer, 'settings_3')
stats_simv_vg_selected.to_excel(writer, 'selected')
writer.save()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_simv_novg_all.xlsx'))
stats_measurements_simv_novg_all.to_excel(writer, 'measurements')
stats_measurements_simv_novg_all_2.to_excel(writer, 'measurements_2')
stats_measurements_simv_novg_all_3.to_excel(writer, 'measurements_3')
stats_settings_simv_novg_all.to_excel(writer, 'settings')
stats_settings_simv_novg_all_2.to_excel(writer, 'settings_2')
stats_settings_simv_novg_all_3.to_excel(writer, 'settings_3')
stats_simv_novg_selected.to_excel(writer, 'selected')
writer.save()

### Inferential statistics of comparing ` group medians`  of `aggregate ventilator parameters` between  SIMV- VG and SIMV no VG groups

The aggregation method for the individual recordings is mean and SD for parametrically distributed parameters and median and IQR for non-parametrically distributed parameters

### PIP and VTemand

##### PIP
Parameterically distributed - use `means` of the individual recordings

In [ ]:
PIP_simv_vg = stats_measurements_simv_vg_all.swaplevel(0,1).loc['PIP']
PIP_simv_vg.head()

In [ ]:
PIP_simv_novg = stats_measurements_simv_novg_all.swaplevel(0,1).loc['PIP']
PIP_simv_novg.head()

In [ ]:
PIP_simv_vg['mean'].describe()

In [ ]:
PIP_simv_novg['mean'].describe()

In [ ]:
PIP_mw_stats = stats.mannwhitneyu(PIP_simv_vg['mean'], PIP_simv_novg['mean'])
PIP_mw_stats

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV-VG', 'SIMV']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([PIP_simv_vg['mean'].dropna(), PIP_simv_novg['mean'].dropna()], widths = 0.5,
        whis = 'range', showfliers = True,showmeans = True, 
        meanprops = meanpointprops, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 40)
ax.set_xlabel('Ventilation mode', size = 14)
ax.set_ylabel('PIP (cmH$_2$O)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'SIMV_VG_noVG_PIP', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

##### VTemand
Parameterically distributed - use `means` of the individual recordings

In [ ]:
VTemand_simv_vg = stats_measurements_simv_vg_all.swaplevel(0,1).loc['VTemand_resp_kg']
VTemand_simv_vg.head()

In [ ]:
VTemand_simv_novg = stats_measurements_simv_novg_all.swaplevel(0,1).loc['VTemand_resp_kg']
VTemand_simv_novg.head()

In [ ]:
VTemand_simv_vg['mean'].describe()

In [ ]:
VTemand_simv_novg['mean'].describe()

In [ ]:
VTemand_mw_stats = stats.mannwhitneyu(VTemand_simv_vg['mean'].dropna(), VTemand_simv_novg['mean'].dropna())
VTemand_mw_stats

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV-VG', 'SIMV']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([VTemand_simv_vg['mean'].dropna(), VTemand_simv_novg['mean'].dropna()], widths = 0.5,
        whis = 'range', showfliers = True,showmeans = True, 
        meanprops = meanpointprops, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 16)
ax.set_xlabel('Ventilation mode', size = 14)
ax.set_ylabel('VTemand (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'SIMV_VG_noVG_VTemand_resp', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

### MVspon
The infants' contribution (in %) to the total minute volume

Non-parametrically distributed, used medians

In [ ]:
MVspon_simv_vg = stats_measurements_simv_vg_all.swaplevel(0,1).loc['MVspon']

MVspon_simv_novg = stats_measurements_simv_novg_all.swaplevel(0,1).loc['MVspon']

In [ ]:
# Percentage of spontaneous breathing

MVspon_simv_vg['median'].describe()

In [ ]:
MVspon_simv_novg['median'].describe()

In [ ]:
MVspon_mw_stats = stats.mannwhitneyu(MVspon_simv_vg['median'].dropna(), MVspon_simv_novg['median'].dropna())
MVspon_mw_stats

### MVresp
The ventilator inflations' contribution (in %) to the total minute volume

Non-parametrically distributed, used medians

In [ ]:
MVresp_simv_vg = stats_measurements_simv_vg_all.swaplevel(0,1).loc['MVresp']

In [ ]:
MVresp_simv_novg = stats_measurements_simv_novg_all.swaplevel(0,1).loc['MVresp']

In [ ]:
# Percentage of spontaneous breathing

MVresp_simv_vg['median'].describe()

In [ ]:
MVresp_simv_novg['median'].describe()

In [ ]:
MVresp_mw_stats = stats.mannwhitneyu(MVresp_simv_vg['median'].dropna(), MVresp_simv_novg['median'].dropna())
MVresp_mw_stats

### Other ventilator parameters (FiO2, MAP, RR, MV, PEEP)

##### FiO2
Non-parameterically distributed - use `medians` of the individual recordings

In [ ]:
FiO2_simv_vg = stats_settings_simv_vg_all.swaplevel(0,1).loc['FiO2_set']
FiO2_simv_novg = stats_settings_simv_novg_all.swaplevel(0,1).loc['FiO2_set']

In [ ]:
FiO2_simv_vg['median'].describe()

In [ ]:
FiO2_simv_novg['median'].describe()

In [ ]:
FiO2_mw_stats = stats.mannwhitneyu(FiO2_simv_vg['median'], FiO2_simv_novg['median'])
FiO2_mw_stats

##### MAP
Parameterically distributed - use `means` of the individual recordings

In [ ]:
MAP_simv_vg = stats_measurements_simv_vg_all.swaplevel(0,1).loc['MAP']
MAP_simv_novg = stats_measurements_simv_novg_all.swaplevel(0,1).loc['MAP']

In [ ]:
MAP_simv_vg['mean'].describe()

In [ ]:
MAP_simv_novg['mean'].describe()

In [ ]:
MAP_mw_stats = stats.mannwhitneyu(MAP_simv_vg['mean'], MAP_simv_novg['mean'])
MAP_mw_stats

##### RR_set
Parameterically distributed - use `means` of the individual recordings

In [ ]:
RRset_simv_vg = stats_settings_simv_vg_all.swaplevel(0,1).loc['RR_set']
RRset_simv_novg = stats_settings_simv_novg_all.swaplevel(0,1).loc['RR_set']

In [ ]:
RRset_simv_vg['mean'].describe()

In [ ]:
RRset_simv_novg['mean'].describe()

In [ ]:
RRset_mw_stats = stats.mannwhitneyu(RRset_simv_vg['mean'], RRset_simv_novg['mean'])
RRset_mw_stats

##### MV
Parameterically distributed - use `means` of the individual recordings

In [ ]:
MV_simv_vg = stats_measurements_simv_vg_all.swaplevel(0,1).loc['MV_kg']
MV_simv_novg = stats_measurements_simv_novg_all.swaplevel(0,1).loc['MV_kg']

In [ ]:
MV_simv_vg['mean'].describe()

In [ ]:
MV_simv_novg['mean'].describe()

In [ ]:
MV_mw_stats = stats.mannwhitneyu(MV_simv_vg['mean'], MV_simv_novg['mean'])
MV_mw_stats

##### PEEP
Parameterically distributed - use `means` of the individual recordings

In [ ]:
PEEP_simv_vg = stats_measurements_simv_vg_all.swaplevel(0,1).loc['PEEP']
PEEP_simv_novg = stats_measurements_simv_novg_all.swaplevel(0,1).loc['PEEP']

In [ ]:
PEEP_simv_vg['mean'].describe()

In [ ]:
PEEP_simv_novg['mean'].describe()

In [ ]:
PEEP_mw_stats = stats.mannwhitneyu(PEEP_simv_vg['mean'], PEEP_simv_novg['mean'])
PEEP_mw_stats

### Inferential statistics of comparing ` group medians`  avereage (mean or median as appropriate) ventilator parameters` between  VG and no VG groups

In [ ]:
def iqr(ser):
    return round(ser.quantile(0.25), 2), round(ser.quantile(0.75), 2)  

In [ ]:
def minmax(ser):
    return round(ser.min(), 2), round(ser.max(), 2)

In [ ]:
pars_1 = ['PIP', 'VTemand_resp_kg',  'MAP',  'MV_kg', ] # measurements and mean as aggregate
pars_2 = ['Leak', ] # measurements and median as aggregate
pars_3 = ['RR_set', 'PEEP_set'] # settings and mean as aggregate
pars_4 = ['FiO2_set',] # settings and median as aggregate

group_median_vg = []
group_iqr_vg = []
group_range_vg = []
group_median_novg = []
group_iqr_novg = []
group_range_novg = []
mw_test = []

for par in pars_1:
    vg_mean = stats_measurements_simv_vg_all.swaplevel(0,1).loc[par]['mean']
    group_median_vg.append(vg_mean.median())
    group_iqr_vg.append(iqr(vg_mean))
    group_range_vg.append(minmax(vg_mean))
    
    novg_mean = stats_measurements_simv_novg_all.swaplevel(0,1).loc[par]['mean']
    group_median_novg.append(novg_mean.median())
    group_iqr_novg.append(iqr(novg_mean))
    group_range_novg.append(minmax(novg_mean))
    
    mw_test.append(stats.mannwhitneyu(vg_mean.dropna(), novg_mean.dropna())[1])
    
for par in pars_2:
    vg_median = stats_measurements_simv_vg_all.swaplevel(0,1).loc[par]['median']
    group_median_vg.append(vg_median.median())
    group_iqr_vg.append(iqr(vg_median))
    group_range_vg.append(minmax(vg_median))
    
    novg_median = stats_measurements_simv_novg_all.swaplevel(0,1).loc[par]['median']
    group_median_novg.append(novg_median.median())
    group_iqr_novg.append(iqr(novg_median))
    group_range_novg.append(minmax(novg_median))
    
    mw_test.append(stats.mannwhitneyu(vg_median.dropna(), novg_median.dropna())[1])
    
for par in pars_3:
    vg_mean = stats_settings_simv_vg_all.swaplevel(0,1).loc[par]['mean']
    group_median_vg.append(vg_mean.median())
    group_iqr_vg.append(iqr(vg_mean))
    group_range_vg.append(minmax(vg_mean))
    
    novg_mean = stats_settings_simv_novg_all.swaplevel(0,1).loc[par]['mean']
    group_median_novg.append(novg_mean.median())
    group_iqr_novg.append(iqr(novg_mean))
    group_range_novg.append(minmax(novg_mean))
    
    mw_test.append(stats.mannwhitneyu(vg_mean.dropna(), novg_mean.dropna())[1])
      
for par in pars_4:
    vg_median = stats_settings_simv_vg_all.swaplevel(0,1).loc[par]['median']
    group_median_vg.append(vg_median.median())
    group_iqr_vg.append(iqr(vg_median))
    group_range_vg.append(minmax(vg_median))
    
    novg_median = stats_settings_simv_novg_all.swaplevel(0,1).loc[par]['median']
    group_median_novg.append(novg_median.median())
    group_iqr_novg.append(iqr(novg_median))
    group_range_novg.append(minmax(novg_median))
    
    mw_test.append(stats.mannwhitneyu(vg_median.dropna(), novg_median.dropna())[1])
    
pars = pars_1 + pars_2 + pars_3 + pars_4

VG_stats = round(DataFrame([group_median_vg, group_median_novg, group_iqr_vg, 
                            group_iqr_novg, group_range_vg, group_range_novg]).T, 2)

VG_stats.columns =      ['VG (group median)', 'noVG (group median)',
                         'VG (group iqr)', 'noVG (group iqr)',
                         'VG (group range)', 'noVG (group range)']

VG_stats['p value'] = mw_test

VG_stats.index = pars
VG_stats.sort_values(by = 'p value', inplace = True)

In [ ]:
VG_stats

### Correction for multiple testing using false discovery rate (Benjamini-Hochberg)

One good technique for controlling the false discovery rate was briefly mentioned by Simes (1986) and developed in detail by Benjamini and Hochberg (1995). Put the individual P values in order, from smallest to largest. The smallest P value has a rank of i=1, then next smallest has i=2, etc. Compare each individual P value to its Benjamini- Hochberg critical value, (i/m)Q, where i is the rank, m is the total number of tests, and Q is the false discovery rate you choose. The largest P value that has P<(i/m)Q is significant, and all of the P values smaller than it are also significant, even the ones that aren’t less than their Benjamini-Hochberg critical value.

In [ ]:
def fdr(ser, rate = 0.05):
    '''
    
    '''
    result = []
    m = len(ser)
    ser_sorted = ser.sort_values()
    for i in range(m):
        result.append(   round(((i+1) / m * rate), 3))

    return result

In [ ]:
p_values = VG_stats[['p value']].copy()
p_values.columns = ['p value']

In [ ]:
p_values

In [ ]:
p_values_fdr = fdr(p_values['p value'])
p_values['fdr'] =  p_values_fdr
p_values['corrected p'] = p_values['p value'] * (0.05 / p_values['fdr'])
p_values.drop('p value', axis = 1, inplace = True)

In [ ]:
VG_stats_corrected = VG_stats.join(p_values)
VG_stats_corrected

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'inferential_stats.xlsx'))
VG_stats.to_excel(writer, 'VG_stats')
VG_stats_corrected.to_excel(writer, 'VG_stats_corrected')
writer.save()

### Descriptive stats on VTset, VTdiff and Pdiff in SIMV-VG recordings

##### VTset 

In [ ]:
VTset_simv_vg = stats_settings_simv_vg_all.swaplevel(0,1).loc['VG_set_kg']

In [ ]:
VTset_simv_vg['mean'].describe()

#### VTdiff = VTemand_kg - VG_set_kg

In [ ]:
VTdiff_simv_vg = stats_measurements_simv_vg_all.swaplevel(0,1).loc['VTdiff']

In [ ]:
VTdiff_simv_vg;

In [ ]:
VTdiff_simv_vg['mean'].describe()

In [ ]:
VTdiff_simv_vg['mean']

In [ ]:
VTdiff_simv_vg['mean'][VTdiff_simv_vg['mean'] > 1 ]

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV-VG', ]

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([VTdiff_simv_vg['mean'].dropna()], widths = 0.5,
        whis = 'range', showfliers = True,showmeans = True, 
        meanprops = meanpointprops, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(-2, 6)
ax.set_xlabel('Ventilation mode', size = 14)
ax.set_ylabel('VTdiff (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'SIMV_VG_noVG_VTdiff', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

#### Pdiff = Pmax - PIP

In [ ]:
Pdiff_simv_vg = stats_measurements_simv_vg_all.swaplevel(0,1).loc['Pdiff']

In [ ]:
Pdiff_simv_vg;

In [ ]:
Pdiff_simv_vg['mean'].describe()

In [ ]:
Pdiff_simv_vg = stats_measurements_simv_vg_all.swaplevel(0,1).loc['Pdiff']

In [ ]:
Pdiff_simv_vg;

In [ ]:
Pdiff_simv_vg['mean'].describe()

In [ ]:
Pdiff_simv_vg['mean'][Pdiff_simv_vg['mean'] <5 ]

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV-VG', ]

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([Pdiff_simv_vg['mean'].dropna()], widths = 0.5,
        whis = 'range', showfliers = True,showmeans = True, 
        meanprops = meanpointprops, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(-5, 30)
ax.set_xlabel('Ventilation mode', size = 14)
ax.set_ylabel('Pdiff (cmH$_2$O)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'SIMV_VG_noVG_Pdiff', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

### Correlation between MVresp and PIP

In [ ]:
MVresp_simv_vg;

In [ ]:
stats.pearsonr(100 - MVresp_simv_vg['mean'], PIP_simv_vg['mean'])

In [ ]:
def corr_pearson(x, y):

    '''
    input: two numeric arrays of the same size

    returns: a tuple of 
    1. Pearson's correlation coefficient: r
    2. low and high 95% confidence intervals or r (two values)
    3. Coefficient of determination: r^2
    4. p-value of correlation

    '''
    
    assert len(x) == len(y)
    
    r, p = stats.pearsonr(x, y)
    f = 0.5*np.log((1+r)/(1-r))
    se = 1/np.sqrt(len(x)-3)
    ucl = f + 1.96 * se
    lcl = f - 1.96 * se

    lcl = (np.exp(2*lcl) - 1) / (np.exp(2*lcl) + 1)
    ucl = (np.exp(2*ucl) - 1) / (np.exp(2*ucl) + 1)

    return r , lcl, ucl , r*r, p

In [ ]:
dpi = 300
filetype = 'jpg'

fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = 100 - MVresp_simv_vg['mean']
y = PIP_simv_vg['mean']

plt.scatter(x, y, color = 'black', marker = 'o', s = 30, )

plt.xlim([-10, 100])
plt.ylim([0, 40])
plt.xlabel('Proportion of spontaneous breathing (%)', fontsize  = 14)
plt.ylabel('PIP (cmH$_2$O)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)

# Polynomial Coefficients
coeffs = np.polyfit(x.values.astype('float'), 
                    y.values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(x, y)

# print the equation on the graph area 
text = 'y=%.2fx + %.2f\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(45 , 33 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MVspon_pc_PIP',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi = 300
filetype = 'jpg'

fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = 100 - MVresp_simv_vg['mean']
y = VTemand_simv_vg['mean']

plt.scatter(x, y, color = 'black', marker = 'o', s = 30, )

plt.xlim([-10, 100])
plt.ylim([0, 12])
plt.xlabel('Proportion of spontaneous breathing (%)', fontsize  = 14)
plt.ylabel('VTemand (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MVspon_pc_VTemand',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1,);

In [ ]:
dpi = 300
filetype = 'jpg'

fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = 100 - MVresp_simv_vg['mean']
y = VTdiff_simv_vg['mean']

plt.scatter(x, y, color = 'black', marker = 'o', s = 30, )

plt.xlim([-10, 100])
plt.ylim([-1, 7])
plt.xlabel('Proportion of spontaneous breathing (%)', fontsize  = 14)
plt.ylabel('VTdiff (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)

# Polynomial Coefficients
coeffs = np.polyfit(x.values.astype('float'), 
                    y.values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(x, y)

# print the equation on the graph area 
text = 'y=%.2fx + %.2f\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(45 , 5.5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MVspon_pc_VTdiff',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, );

In [ ]:
dpi = 300
filetype = 'jpg'

fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = PIP_simv_vg['mean']
y = VTemand_simv_vg['mean']

plt.scatter(x, y, color = 'black', marker = 'o', s = 30, )

plt.xlim([0, 50])
plt.ylim([0, 10])
plt.xlabel('PIP (mbar)', fontsize = 14)
plt.ylabel('VTemand (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)

# Polynomial Coefficients
coeffs = np.polyfit(x.values.astype('float'), 
                    y.values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(x, y)

# print the equation on the graph area 
text = 'y=%.2fx + %.2f\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(25 , 8 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'PIP_VTemand',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, );

### Compare those with >50% MVspon_pc and those with <50% MVspon_pcm

In [ ]:
MVspon_simv_vg['mean'].describe()

In [ ]:
MVspon_over_50_pc = sorted(MVspon_simv_vg[MVspon_simv_vg['mean'] >50].index)

MVspon_over_50_pc

In [ ]:
MVspon_under_50_pc = sorted(MVspon_simv_vg[MVspon_simv_vg['mean'] <= 50].index)

MVspon_under_50_pc

In [ ]:
MVspon_over_50_pc_frame = stats_measurements_simv_vg_all.unstack(level = 1).reindex(MVspon_over_50_pc)
MVspon_under_50_pc_frame = stats_measurements_simv_vg_all.unstack(level = 1).reindex(MVspon_under_50_pc)

In [ ]:
MVspon_over_50_pc_frame['mean']['PIP'].describe()

In [ ]:
MVspon_under_50_pc_frame['mean']['PIP'].describe()

In [ ]:
MVspon_50_pc_mw_stats = stats.mannwhitneyu(MVspon_over_50_pc_frame['mean']['PIP'], 
                                           MVspon_under_50_pc_frame['mean']['PIP'])
MVspon_50_pc_mw_stats

### Analyse separately those recordings where the PIP was <10 cmH$_2$O 

In [ ]:
low_PIP = sorted(PIP_simv_vg[PIP_simv_vg['mean'] < 10 ].index)

In [ ]:
not_low_PIP = sorted(PIP_simv_vg[PIP_simv_vg['mean'] >= 10 ].index)

In [ ]:
len(low_PIP), len(not_low_PIP)

In [ ]:
low_PIP

In [ ]:
not_low_PIP

In [ ]:
low_PIP_stats_frame = stats_measurements_simv_vg_all.unstack(level = 1).reindex(low_PIP)
not_low_PIP_stats_frame = stats_measurements_simv_vg_all.unstack(level = 1).reindex(not_low_PIP)

In [ ]:
low_PIP_stats_frame

In [ ]:
not_low_PIP_stats_frame

In [ ]:
low_PIP_clinical_1 = clin_df_HIE.reindex(low_PIP)
low_PIP_clinical_2 = clin_df_HIE_2.reindex(low_PIP)
not_low_PIP_clinical_1 = clin_df_HIE.reindex(not_low_PIP)
not_low_PIP_clinical_2 = clin_df_HIE_2.reindex(not_low_PIP)

In [ ]:
low_PIP_clinical_1;

In [ ]:
low_PIP_clinical_2;

In [ ]:
not_low_PIP_clinical_1;

In [ ]:
not_low_PIP_clinical_2;

### Comparison of sedation and muscle relaxation between "low PIP" and "no low PIP" the groups

In [ ]:
low_PIP_clinical_2['Sedatives'].value_counts(dropna = False)

- Sedation: 11
- No sedation: 2

In [ ]:
not_low_PIP_clinical_2['Sedatives'].value_counts(dropna = False)

- Sedation: 10
- No sedation: 5

In [ ]:
low_PIP_clinical_2['Muscle relaxant'].value_counts(dropna = False)

In [ ]:
not_low_PIP_clinical_2['Muscle relaxant'].value_counts(dropna = False)

In [ ]:
low_PIP_clinical_2['Anticonvulsant'].value_counts(dropna = False)

- No: 8
- Phenobarbital: 5

In [ ]:
not_low_PIP_clinical_2['Anticonvulsant'].value_counts(dropna = False)

- No: 11
- Phenobarbital: 4

### Comparison of sedation and muscle relaxation between "low PIP" and "no low PIP" the groups

In [ ]:
low_PIP_stats_frame['mean']['PIP']

In [ ]:
not_low_PIP_stats_frame['mean']['PIP']

In [ ]:
low_PIP_stats_frame['mean']['VTemand_resp_kg']

In [ ]:
not_low_PIP_stats_frame['mean']['VTemand_resp_kg']

In [ ]:
low_PIP_VTemand_resp_mw_stats = stats.mannwhitneyu(low_PIP_stats_frame['mean']['VTemand_resp_kg'], 
                                             not_low_PIP_stats_frame['mean']['VTemand_resp_kg'])
low_PIP_VTemand_resp_mw_stats

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['PIP <10', 'PIP >10']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([low_PIP_stats_frame['mean']['VTemand_resp_kg'], 
             not_low_PIP_stats_frame['mean']['VTemand_resp_kg']], widths = 0.5, whis = 'range', 
            showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)
plt.text(1.25, 9, 'p = 0.006')

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 10)
ax.set_xlabel('cmH$_2$O', size = 14)
ax.set_ylabel('VTemand (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'low_PIP_not_low_PIP_VTemand_resp', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, );

In [ ]:
low_PIP_stats_frame['mean']['VTdiff']

In [ ]:
not_low_PIP_stats_frame['mean']['VTdiff']

In [ ]:
low_PIP_VTdiff_mw_stats = stats.mannwhitneyu(low_PIP_stats_frame['mean']['VTdiff'], 
                                             not_low_PIP_stats_frame['mean']['VTdiff'])
low_PIP_VTdiff_mw_stats

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['PIP <10', 'PIP >10']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([low_PIP_stats_frame['mean']['VTdiff'], not_low_PIP_stats_frame['mean']['VTdiff']], 
            widths = 0.5, whis = 'range', 
            showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)
plt.text(1.25, 5.25, 'p < 0.0001')

ax.set_xticklabels(xticklabels)
ax.set_ylim(-1, 6)
ax.set_xlabel('cmH$_2$O', size = 14)
ax.set_ylabel('VTdiff (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'low_PIP_not_low_PIP_VTdiff', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

### Analyse blood gases

In [ ]:
blood_gases_simv_vg = {case: blood_gases[case] for case in blood_gases.keys()
                       if case in simv_vg }
len(blood_gases_simv_vg)

In [ ]:
blood_gases_simv_novg = {case: blood_gases[case] for case in blood_gases.keys()
                       if case in simv_novg }
len(blood_gases_simv_novg)

In [ ]:
gases_vg = {}
for case in sorted(blood_gases_simv_vg):
    #print(case)
    if len(blood_gases_simv_vg[case].index) > 1:
        gases_vg[case] = DataFrame([float(blood_gases_simv_vg[case].iloc[0]['pH']),
                                    float(blood_gases_simv_vg[case].iloc[-1]['pH']),
                                    float(blood_gases_simv_vg[case].iloc[0]['pCO2']),
                                    float(blood_gases_simv_vg[case].iloc[-1]['pCO2']),
                                    float(blood_gases_simv_vg[case].iloc[0].get(['ABE'], np.nan)),
                                    float(blood_gases_simv_vg[case].iloc[-1].get(['ABE'], np.nan))]).T
        gases_vg[case].columns = ['pH_before', 'pH_after', 'pCO2_before', 'pCO2_after', 
                                  'BE_before', 'BE_after'] 

gases_vg = pd.concat(gases_vg)
gases_vg.index = gases_vg.index.droplevel(level = 1)

gases_vg['pH_diff'] = gases_vg['pH_after'] - gases_vg['pH_before']
gases_vg['pCO2_diff'] = gases_vg['pCO2_after'] - gases_vg['pCO2_before']
gases_vg['BE_diff'] = gases_vg['BE_after'] - gases_vg['BE_before']

In [ ]:
gases_novg = {}
for case in sorted(blood_gases_simv_novg):
    # print(case)
    if len(blood_gases_simv_novg[case].index) > 1:
        gases_novg[case] = DataFrame([float(blood_gases_simv_novg[case].iloc[0]['pH']),
                                      float(blood_gases_simv_novg[case].iloc[-1]['pH']),
                                      float(blood_gases_simv_novg[case].iloc[0]['pCO2']),
                                      float(blood_gases_simv_novg[case].iloc[-1]['pCO2']),
                                      float(blood_gases_simv_novg[case].iloc[0].get(['ABE'], np.nan)),
                                      float(blood_gases_simv_novg[case].iloc[-1].get(['ABE'], np.nan))]).T
        gases_novg[case].columns = ['pH_before', 'pH_after', 'pCO2_before', 'pCO2_after', 
                                    'BE_before', 'BE_after'] 

gases_novg = pd.concat(gases_novg)
gases_novg.index = gases_novg.index.droplevel(level = 1)

gases_novg['pH_diff'] = gases_novg['pH_after'] - gases_novg['pH_before']
gases_novg['pCO2_diff'] = gases_novg['pCO2_after'] - gases_novg['pCO2_before']
gases_novg['BE_diff'] = gases_novg['BE_after'] - gases_novg['BE_before']

In [ ]:
gases_vg.info()

In [ ]:
gases_vg

In [ ]:
gases_novg.info()

In [ ]:
gases_novg

In [ ]:
gases_vg.describe()

In [ ]:
gases_novg.describe()

#### Blood gases before transport

In [ ]:
stats.mannwhitneyu(gases_vg['pCO2_before'],gases_novg['pCO2_before'])

In [ ]:
stats.mannwhitneyu(gases_vg['pH_before'],gases_novg['pH_before'])

In [ ]:
stats.mannwhitneyu(gases_vg['BE_before'],gases_novg['BE_before'])

#### Blood gases after transport

In [ ]:
stats.mannwhitneyu(gases_vg['pCO2_after'],gases_novg['pCO2_after'])

In [ ]:
stats.mannwhitneyu(gases_vg['pH_after'],gases_novg['pH_after'])

In [ ]:
stats.mannwhitneyu(gases_vg['BE_after'],gases_novg['BE_after'])

In [ ]:
# Arrival blood gases with < 35 mmHg (4.65 kPa)
gases_vg['pCO2_after'][gases_vg['pCO2_after'] < 35] 

In [ ]:
# Arrival blood gases with < 35 mmHg (4.65 kPa)
gases_novg['pCO2_after'][gases_novg['pCO2_after'] < 35] 

In [ ]:
# Arrival blood gases with < 30 mmHg (4 kPa)
gases_vg['pCO2_after'][gases_vg['pCO2_after'] < 30] 

In [ ]:
# Arrival blood gases with < 30 mmHg (4 kPa)
gases_novg['pCO2_after'][gases_novg['pCO2_after'] < 30] 

In [ ]:
gases_vg['pCO2_after']

In [ ]:
gases_novg['pCO2_after']

In [ ]:
gases_vg['pCO2_after'].describe()

In [ ]:
gases_novg['pCO2_after'].describe()

#### Difference in gases before and after the transport

In [ ]:
stats.mannwhitneyu(gases_vg['pCO2_diff'], gases_novg['pCO2_diff'])

In [ ]:
stats.mannwhitneyu(gases_vg['pH_diff'], gases_novg['pH_diff'])

In [ ]:
stats.mannwhitneyu(gases_vg['BE_diff'], gases_novg['BE_diff'])

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
ax.plot([1, 2], [gases_vg['pCO2_before'], gases_vg['pCO2_after']], color = 'black', marker = 'o')
ax.set_xlim(0.5, 2.5)
ax.set_ylim(0, 110)
ax.set_xticks([1,2])
ax.set_xticklabels(['before transfer', 'after transfer'], size = 14)
ax.set_ylabel(r'pCO$_2$ (mmHg)', size = 14)
ax.set_title('', size = 14)
ax.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,'_simv_vg_CO2_before_after', 'jpg'),
    dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches= 'tight', pad_inches=0.1);

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
ax.plot([1, 2], [gases_novg['pCO2_before'], gases_novg['pCO2_after']], color = 'black', marker = 'o')
ax.set_xlim(0.5, 2.5)
ax.set_ylim(0, 100)
ax.set_xticks([1,2])
ax.set_xticklabels(['before transfer', 'after transfer'], size = 14)
ax.set_ylabel(r'pCO$_2$ (mmHg)', size = 14)
ax.set_title('', size = 14)
ax.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,'_simv_novg_CO2_before_after', 'jpg'),
    dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches= 'tight', pad_inches=0.1);

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
ax.plot([1, 2], [gases_vg['pH_before'], gases_vg['pH_after']], color = 'black', marker = 'o')
ax.set_xlim(0.5, 2.5)
ax.set_ylim(6, 7.5)
ax.set_xticks([1,2])
ax.set_xticklabels(['before transfer', 'after transfer'], size = 14)
ax.set_ylabel(r'pH', size = 14)
ax.set_title('', size = 14)
ax.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,'_simv_vg_pH_before_after', 'jpg'),
    dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches= 'tight', pad_inches=0.1);

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
ax.plot([1, 2], [gases_novg['pH_before'], gases_novg['pH_after']], color = 'black', marker = 'o')
ax.set_xlim(0.5, 2.5)
ax.set_ylim(6, 7.5)
ax.set_xticks([1,2])
ax.set_xticklabels(['before transfer', 'after transfer'], size = 14)
ax.set_ylabel(r'pH', size = 14)
ax.set_title('', size = 14)
ax.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,'_simv_novg_pH_before_after', 'jpg'),
    dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches= 'tight', pad_inches=0.1);

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
ax.plot([1, 2], [gases_vg['BE_before'], gases_vg['BE_after']], color = 'black', marker = 'o')
ax.set_xlim(0.5, 2.5)
ax.set_ylim(0, -30)
ax.set_xticks([1,2])
ax.set_xticklabels(['before transfer', 'after transfer'], size = 14)
ax.set_ylabel(r'BE (mmol/L)', size = 14)
ax.set_title('', size = 14)
ax.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,'_simv_vg_BE_before_after', 'jpg'),
    dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches= 'tight', pad_inches=0.1);

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
ax.plot([1, 2], [gases_novg['BE_before'], gases_novg['BE_after']], color = 'black', marker = 'o')
ax.set_xlim(0.5, 2.5)
ax.set_ylim(0, -30)
ax.set_xticks([1,2])
ax.set_xticklabels(['before transfer', 'after transfer'], size = 14)
ax.set_ylabel(r'BE (mmol/L)', size = 14)
ax.set_title('', size = 14)
ax.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,'_simv_novg_BE_before_after', 'jpg'),
    dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches= 'tight', pad_inches=0.1);

#### Consider those recordings where the average PIP is less than 10. 

In [ ]:
gases_low_PIP = gases_vg.reindex(low_PIP)
gases_not_low_PIP = gases_vg.reindex(not_low_PIP)

In [ ]:
gases_low_PIP['pCO2_after'].describe()

In [ ]:
gases_not_low_PIP['pCO2_after'].describe()

In [ ]:
stats.mannwhitneyu(gases_low_PIP['pCO2_after'], gases_not_low_PIP['pCO2_after'])

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['PIP <10', 'PIP >10']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([gases_low_PIP['pCO2_after'].dropna(), 
             gases_not_low_PIP['pCO2_after'].dropna()], widths = 0.5, whis = 'range', 
            showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

plt.text(1.25, 100, 'p = 0.11')
ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 110)
ax.set_xlabel('cmH$_2$O', size = 14)
ax.set_ylabel('pCO2 after transport (mmHg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'low_PIP_not_low_PIP_pCO2_after', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

In [ ]:
gases_low_PIP['pCO2_diff'].describe()

In [ ]:
gases_not_low_PIP['pCO2_diff'].describe()

In [ ]:
stats.mannwhitneyu(gases_low_PIP['pCO2_diff'], gases_not_low_PIP['pCO2_diff'])

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['PIP <10', 'PIP >10']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([gases_low_PIP['pCO2_diff'].dropna(), 
             gases_not_low_PIP['pCO2_diff'].dropna()], widths = 0.5, whis = 'range', 
            showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(-80, 80)
ax.set_xlabel('cmH$_2$O', size = 14)
ax.set_ylabel('Change in pCO2 during transport (mmHg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'low_PIP_not_low_PIP_pCO2_diff', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

#### pCO2 before, after and diff versus MVspon_pc

In [ ]:
MVspon_pc = 100 - MVresp_simv_vg['mean']
MV_CO2 = DataFrame([MVspon_pc, gases_vg['pCO2_before'], gases_vg['pCO2_after'], gases_vg['pCO2_diff'] ]).T
MV_CO2.columns = ['MVspon_pc', 'pCO2_before', 'pCO2_after', 'pCO2_diff']
MV_CO2.dropna(inplace = True)

In [ ]:
MV_CO2;

In [ ]:
dpi = 300
filetype = 'jpg'

fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = MV_CO2['MVspon_pc']
y = MV_CO2['pCO2_before']

plt.scatter(x, y, color = 'black', marker = 'o', s = 30, )

plt.xlim([-10, 100])
plt.ylim([0, 150])
plt.xlabel('Proportion of spontaneous breathing (%)', fontsize  = 14)
plt.ylabel('pCO2 before transport (mmHg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)

# Polynomial Coefficients
coeffs = np.polyfit(x.values.astype('float'), 
                    y.values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(x, y)

# print the equation on the graph area 
text = 'y=%.2fx + %.2f\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(45 , 120 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MVspon_pc_pCO2_before',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi = 300
filetype = 'jpg'

fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = MV_CO2['MVspon_pc']
y = MV_CO2['pCO2_after']

plt.scatter(x, y, color = 'black', marker = 'o', s = 30, )

plt.xlim([-10, 100])
plt.ylim([0, 150])
plt.xlabel('Proportion of spontaneous breathing (%)', fontsize  = 14)
plt.ylabel('pCO$_2$ (mmHg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)

# Polynomial Coefficients
coeffs = np.polyfit(x.values.astype('float'), 
                    y.values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(x, y)

# print the equation on the graph area 
text = 'y=%.2fx + %.2f\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(45 , 120 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MVspon_pc_pCO2_after',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi = 300
filetype = 'jpg'

fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = MV_CO2['MVspon_pc']
y = MV_CO2['pCO2_diff']

plt.scatter(x, y, color = 'black', marker = 'o', s = 30, )

plt.xlim([-10, 100])
plt.ylim([-100, 100])
plt.xlabel('Proportion of spontaneous breathing (%)', fontsize  = 14)
plt.ylabel('Difference in pCO2 before and after transport (mmHg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MVspon_pc_pCO2_diff',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1,);

### pH

#### pH before the transport

In [ ]:
gases_vg['pH_before'].describe()

In [ ]:
gases_novg['pH_before'].describe()

In [ ]:
stats.mannwhitneyu(gases_vg['pH_before'],gases_novg['pH_before'])

#### pH after the transport

In [ ]:
gases_vg['pH_after'].describe()

In [ ]:
gases_novg['pH_after'].describe()

In [ ]:
stats.mannwhitneyu(gases_vg['pH_after'],gases_novg['pH_after'])

In [ ]:
# Arrival blood gases with pH < 7.1
gases_vg['pH_after'][gases_vg['pH_after'] < 7.1].sort_values() 

In [ ]:
# Arrival blood gases with pH < 7.1
gases_novg['pH_after'][gases_novg['pH_after'] < 7.1].sort_values() 

#### Difference in pH before and after the transport

In [ ]:
gases_vg.sort_values('pH_diff')

In [ ]:
gases_novg.sort_values('pH_diff')

In [ ]:
gases_vg['pH_diff'].describe()

In [ ]:
gases_novg['pH_diff'].describe()

In [ ]:
stats.mannwhitneyu(gases_vg['pH_diff'],gases_novg['pH_diff'])

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
ax.plot([1, 2], [gases_vg['pH_before'], gases_vg['pH_after']], color = 'black', marker = 'o')
ax.set_xlim(0.5, 2.5)
ax.set_ylim(6.3, 7.5)
ax.set_xticks([1,2])
ax.set_xticklabels(['before transfer', 'after transfer'], size = 14)
ax.set_ylabel(r'pH', size = 14)
ax.set_title('', size = 14)
ax.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,'_simv_vg_pH_before_after', 'jpg'),
    dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches= 'tight', pad_inches=0.1,);

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
ax.plot([1, 2], [gases_novg['pH_before'], gases_novg['pH_after']], color = 'black', marker = 'o')
ax.set_xlim(0.5, 2.5)
ax.set_ylim(6.3, 7.5)
ax.set_xticks([1,2])
ax.set_xticklabels(['before transfer', 'after transfer'], size = 14)
ax.set_ylabel(r'pH', size = 14)
ax.set_title('', size = 14)
ax.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,'_simv_novg_pH_before_after', 'jpg'),
    dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches= 'tight', pad_inches=0.1,);

#### Consider those recordings where the average PIP is less than 10. 

In [ ]:
gases_low_PIP['pH_after'].describe()

In [ ]:
gases_not_low_PIP['pH_after'].describe()

In [ ]:
stats.mannwhitneyu(gases_low_PIP['pH_after'], gases_not_low_PIP['pH_after'])

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['PIP <10', 'PIP >10']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([gases_low_PIP['pH_after'].dropna(), 
             gases_not_low_PIP['pH_after'].dropna()], widths = 0.5, whis = 'range', 
            showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

plt.text(1.25, 7.38, 'p = 0.070')
ax.set_xticklabels(xticklabels)
ax.set_ylim(6.7, 7.45)
ax.set_xlabel('cmH$_2$O', size = 14)
ax.set_ylabel('pH after transport', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'low_PIP_not_low_PIP_pH_after', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

#### pH before, after and difference versus MVspon_pc

In [ ]:
MVspon_pc = 100 - MVresp_simv_vg['mean']
MV_pH = DataFrame([MVspon_pc, gases_vg['pH_before'], gases_vg['pH_after'], gases_vg['pH_diff'] ]).T
MV_pH.columns = ['MVspon_pc', 'pH_before', 'pH_after', 'pH_diff']
MV_pH.dropna(inplace = True)

In [ ]:
MV_pH;

In [ ]:
dpi = 300
filetype = 'jpg'

fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = MV_pH['MVspon_pc']
y = MV_pH['pH_before']

plt.scatter(x, y, color = 'black', marker = 'o', s = 30, )

plt.xlim([-10, 100])
plt.ylim([6.3, 7.7])
plt.xlabel('Proportion of spontaneous breathing (%)', fontsize  = 14)
plt.ylabel('pH before transport', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)


fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MVspon_pc_pH_before',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1,);

In [ ]:
dpi = 300
filetype = 'jpg'

fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = MV_pH['MVspon_pc']
y = MV_pH['pH_after']

plt.scatter(x, y, color = 'black', marker = 'o', s = 30, )

plt.xlim([-10, 100])
plt.ylim([6.5, 7.7])
plt.xlabel('Proportion of spontaneous breathing (%)', fontsize  = 14)
plt.ylabel('pH after transport', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)


fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MVspon_pc_pH_after',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1,);

In [ ]:
dpi = 300
filetype = 'jpg'

fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = MV_pH['MVspon_pc']
y = MV_pH['pH_diff']

plt.scatter(x, y, color = 'black', marker = 'o', s = 30, )

plt.xlim([-10, 100])
plt.ylim([-1, 1])
plt.xlabel('Proportion of spontaneous breathing (%)', fontsize  = 14)
plt.ylabel('pH difference before and after transport', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)


fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MVspon_pc_pH_diff',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1,);

In [ ]:
dpi = 300
filetype = 'jpg'

fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = MV_pH['MVspon_pc']
y = MV_pH['pH_after']

plt.scatter(x, y, color = 'black', marker = 'o', s = 30, )

plt.xlim([-10, 100])
plt.ylim([6.5, 7.7])
plt.xlabel('Proportion of spontaneous breathing (%)', fontsize  = 14)
plt.ylabel('pH after transport', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)


fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MVspon_pc_pH_after',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1,);

### Which recordings have no RR measurements?

In [ ]:
RR = []
no_RR = []


for recording in recordings:
    if 'RR' not in data_pars_measurements_ventilated_HIE[recording]:
        no_RR.append(recording)
    else:
        RR.append(recording)

In [ ]:
print(no_RR)

In [ ]:
vent_modes_ventilated_HIE.loc[no_RR].head()

## Figures for paper

### Figure 1

In [ ]:
# Define resolution and file type
dpi = 300
filetype = 'eps'

fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = 100 - MVresp_simv_vg['mean']
y = PIP_simv_vg['mean']

plt.scatter(x, y, color = 'black', marker = 'x', s = 30, )

plt.xlim([-10, 100])
plt.ylim([0, 40])
plt.xlabel('Proportion of spontaneous breathing (%)', fontsize  = 14)
plt.ylabel('PIP (cmH$_2$O)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)

# Polynomial Coefficients
coeffs = np.polyfit(x.values.astype('float'), 
                    y.values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'k--', alpha = 0.7)

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(x, y)

# print the equation on the graph area 
text = 'y=%.2fx + %.2f\nr=%.2f (%.2f - %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(45 , 33 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1);

### Figure 2

#### Figure 2A

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['PIP <10', 'PIP >10']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([low_PIP_stats_frame['mean']['VTemand_resp_kg'], 
             not_low_PIP_stats_frame['mean']['VTemand_resp_kg']], widths = 0.5, whis = 'range', 
            showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)
plt.text(1.25, 9, 'p = 0.006')

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 10)
ax.set_xlabel('cmH$_2$O', size = 14)
ax.set_ylabel('VTemand (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2A', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, );

#### Figure 2B

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['PIP <10', 'PIP >10']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([low_PIP_stats_frame['mean']['VTdiff'], not_low_PIP_stats_frame['mean']['VTdiff']], 
            widths = 0.5, whis = 'range', 
            showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)
plt.text(1.25, 5.25, 'p < 0.0001')

ax.set_xticklabels(xticklabels)
ax.set_ylim(-1, 6)
ax.set_xlabel('cmH$_2$O', size = 14)
ax.set_ylabel('VTdiff (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2B', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

#### Figure 2C

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['PIP <10', 'PIP >10']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([gases_low_PIP['pCO2_after'].dropna(), 
             gases_not_low_PIP['pCO2_after'].dropna()], widths = 0.5, whis = 'range', 
            showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)
plt.text(1.25, 100, 'p = 0.11')

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 110)
ax.set_xlabel('cmH$_2$O', size = 14)
ax.set_ylabel('pCO$_2$ after transport (mmHg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2C', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1);

#### Figure 2D

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['PIP <10', 'PIP >10']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([gases_low_PIP['pH_after'].dropna(), 
             gases_not_low_PIP['pH_after'].dropna()], widths = 0.5, whis = 'range', 
            showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(6.7, 7.4)
ax.set_xlabel('cmH$_2$O', size = 14)
ax.set_ylabel('pH after transport', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2D', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

#### Figure 2 combined

In [ ]:
# Define resolution and file type
dpi = 300
filetype = 'eps'

# Define xticklabels
xticklabels = ['PIP <10', 'PIP >10']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(2,2, figsize = [9,9])
fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, 
                                        hspace=0.3, wspace=0.3)

# Figure 3A
ax[0,0].boxplot([low_PIP_stats_frame['mean']['VTemand_resp_kg'], 
             not_low_PIP_stats_frame['mean']['VTemand_resp_kg']], widths = 0.5, whis = 'range', 
             showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
             boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax[0,0].text(1.25, 9, 'p = 0.006')

ax[0,0].set_xticklabels(xticklabels)
ax[0,0].set_ylim(0, 10)
ax[0,0].set_xlabel('cmH$_2$O', size = 14)
ax[0,0].set_ylabel('VTemand (mL/kg)', size = 14, labelpad = 0)
ax[0,0].tick_params(axis='both', which='major', labelsize=14)
ax[0,0].grid(True)


# Figure 3B
ax[0,1].boxplot([low_PIP_stats_frame['mean']['VTdiff'], not_low_PIP_stats_frame['mean']['VTdiff']], 
            widths = 0.5, whis = 'range', 
            showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)
ax[0,1].text(1.25, 5.25, 'p < 0.0001')

ax[0,1].set_xticklabels(xticklabels)
ax[0,1].set_ylim(-1, 6)
ax[0,1].set_xlabel('cmH$_2$O', size = 14)
ax[0,1].set_ylabel('VTdiff (mL/kg)', size = 14, labelpad = 0)
ax[0,1].tick_params(axis='both', which='major', labelsize=14)
ax[0,1].grid(True)


# Figure 3C
ax[1,0].boxplot([gases_low_PIP['pCO2_after'].dropna(), 
             gases_not_low_PIP['pCO2_after'].dropna()], widths = 0.5, whis = 'range', 
            showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)
ax[1,0].text(1.25, 100, 'p = 0.11')

ax[1,0].set_xticklabels(xticklabels)
ax[1,0].set_ylim(0, 110)
ax[1,0].set_xlabel('cmH$_2$O', size = 14)
ax[1,0].set_ylabel('pCO$_2$ after transport (mmHg)', size = 14, labelpad = 0)
ax[1,0].tick_params(axis='both', which='major', labelsize=14)
ax[1,0].grid(True)

# Figure 3D
ax[1,1].boxplot([gases_low_PIP['pH_after'].dropna(), 
             gases_not_low_PIP['pH_after'].dropna()], widths = 0.5, whis = 'range', 
            showfliers = True, showmeans = True, medianprops=medianprops,  meanprops = meanpointprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax[1,1].text(1.25, 7.38, 'p = 0.070')
ax[1,1].set_xticklabels(xticklabels)
ax[1,1].set_ylim(6.7, 7.45)
ax[1,1].set_xlabel('cmH$_2$O', size = 14)
ax[1,1].set_ylabel('pH after transport', size = 14, labelpad = 0)
ax[1,1].tick_params(axis='both', which='major', labelsize=14)
ax[1,1].grid(True)

fig.text(0.04, 0.92, 'A', fontsize = 16); fig.text(0.50, 0.92, 'B', fontsize = 16)
fig.text(0.04, 0.46, 'C', fontsize = 16); fig.text(0.50, 0.46, 'D', fontsize = 16)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, );

### Supplementary Figure 1

In [ ]:
# Define resolution and file type
dpi = 300
filetype = 'eps'


fig = plt.figure()
fig.set_size_inches(6, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax = fig.add_subplot(1,1,1);

x = MAP_simv_vg['mean']
y = FiO2_simv_vg['mean']

v = MAP_simv_novg['mean']
w = FiO2_simv_novg['mean']

plt.scatter(x, y, color = 'white', edgecolors = 'black', marker = 'o', s = 30, )
plt.scatter(v, w, color = 'black', marker = '+', s = 40, )

plt.xlim([0, 20])
plt.ylim([0, 110])
plt.xlabel('MAP (cmH$_2$O)', fontsize  = 14)
plt.ylabel('FiO$_2$ (%)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
ax.legend().set_visible(False)

a = pd.concat([x, v])
b = pd.concat([y, w])

# Polynomial Coefficients
coeffs = np.polyfit(a.values.astype('float'), 
                    b.values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(a,l(a),'k--', alpha = 0.7)

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(a, b)

# print the equation on the graph area 
text = 'y=%.2fx + %.2f\nr=%.2f (%.2f - %.2f)\np<%.3f' % (result[0], result[1], r, lcl, ucl, 0.001)
plt.text(1 , 90 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_1',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1);

### Supplementary Figure 2

#### Supplementary Figure 2A

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
ax.plot([1, 2], [gases_vg['pCO2_before'], gases_vg['pCO2_after']], color = 'black', marker = 'o')
ax.set_xlim(0.5, 2.5)
ax.set_ylim(0, 110)
ax.set_xticks([1,2])
ax.set_xticklabels(['before transfer', 'after transfer'], size = 14)
ax.set_ylabel(r'pCO$_2$ (mmHg)', size = 14)
ax.set_title('', size = 14)
ax.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,'_simv_vg_CO2_before_after', 'jpg'),
    dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches= 'tight', pad_inches=0.1);

#### Supplementary Figure 2B

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
ax.plot([1, 2], [gases_novg['pCO2_before'], gases_novg['pCO2_after']], color = 'black', marker = 'o')
ax.set_xlim(0.5, 2.5)
ax.set_ylim(0, 100)
ax.set_xticks([1,2])
ax.set_xticklabels(['before transfer', 'after transfer'], size = 14)
ax.set_ylabel(r'pCO$_2$ (mmHg)', size = 14)
ax.set_title('', size = 14)
ax.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,'_simv_novg_CO2_before_after', 'jpg'),
    dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches= 'tight', pad_inches=0.1);

#### Supplementary Figure 2 combined

In [ ]:
# Define resolution and file type
dpi = 300
filetype = 'eps'

# Define xticklabels
xticklabels = ['before', 'after']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
meanpointprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(1,2, figsize = [8,4])
fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, hspace=0.3, wspace=0.3)


# Supplemental Figure 1A
ax[0].plot([1, 2], [gases_vg['pCO2_before'], gases_vg['pCO2_after']], color = 'black', marker = 'o')
ax[0].set_xlim(0.5, 2.5)
ax[0].set_ylim(0, 110)
ax[0].set_xticks([1,2])
ax[0].set_xticklabels(xticklabels, size = 14)
ax[0].set_ylabel(r'pCO$_2$ (mmHg)', size = 14)
ax[0].set_xlabel('transfer', size = 14)
ax[0].grid(True)

# Supplemental Figure 1B
ax[1].plot([1, 2], [gases_novg['pCO2_before'], gases_novg['pCO2_after']], color = 'black', marker = 'o')
ax[1].set_xlim(0.5, 2.5)
ax[1].set_ylim(0, 110)
ax[1].set_xticks([1,2])
ax[1].set_xticklabels(xticklabels, size = 14)
ax[1].set_ylabel('')
ax[1].set_xlabel('transfer', size = 14)
ax[1].grid(True)

fig.text(0.04, 0.92, 'A', fontsize = 16); fig.text(0.50, 0.92, 'B', fontsize = 16)

fig.savefig('%s/%s.%s' % (DIR_WRITE,'Supplementary_Figure_2', filetype),
    dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches= 'tight', pad_inches=0.1);

### Supplementary Table 1

In [ ]:
Suppl_table_1 = stats_simv_vg_selected.copy()

for par in ['MVresp', 'MVspon', 'FiO2_set']:
    Suppl_table_1[par, 'IQR'] = \
    stats_simv_vg_selected[par]['25pc'].apply(lambda x: str(int(round(x, 0)))) + ' - ' + \
    stats_simv_vg_selected[par]['75pc'].apply(lambda x: str(int(round(x, 0))))

for par in ['Pdiff']:
    Suppl_table_1[par, 'IQR'] = \
    stats_simv_vg_selected[par]['25pc'].apply(lambda x: str(int(round(x, 0)))) + ' - ' + \
    stats_simv_vg_selected[par]['75pc'].apply(lambda x: str(int(round(x, 0))))

Suppl_table_1.sort_index(level = 0, axis = 1, ascending = False, inplace = True)

Suppl_table_1;

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'Supplementary_Table_1.xlsx'))
Suppl_table_1.to_excel(writer, 'Suppl_Table_1')
writer.save()

### Supplementary Table 2

In [ ]:
Suppl_table_2 = stats_simv_novg_selected.copy()

for par in ['MVresp', 'MVspon', 'FiO2_set']:
    Suppl_table_2[par, 'IQR'] = \
    stats_simv_novg_selected[par]['25pc'].apply(lambda x: str(int(round(x, 0)))) + ' - ' + \
    stats_simv_novg_selected[par]['75pc'].apply(lambda x: str(int(round(x, 0))))

Suppl_table_2.sort_index(level = 0, axis = 1, ascending = False, inplace = True)

Suppl_table_2;

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'Supplementary_Table_2.xlsx'))
Suppl_table_2.to_excel(writer, 'Suppl_Table_2')
writer.save()